# Hybrid Machine Learning Approach for Stock and Cryptocurrency Forecasting 

---

## Notebook purpose
This notebook documents the full implementation of my dissertation project. The goal is to build and evaluate machine learning and hybrid models to predict next-day price movement trends in:

- **Stock market:** Apple Inc. (AAPL)  
- **Cryptocurrency market:** Bitcoin (BTC-USD)

To predict the next-day closing price and evaluate trend prediction using Directional Accuracy (whether the model correctly predicts up/down movement).

---

## Research aim
To investigate whether a hybrid machine learning approach (Random Forest feature selection + LSTM sequence modelling) can improve prediction performance for both stock and cryptocurrency market trends compared to strong baseline models.

---

## Objectives
This notebook investigates whether a hybrid machine learning approach improves next-day price prediction for:
- Stock: AAPL
- Cryptocurrency: BTC-USD

1. Collect daily OHLCV price data from 2018 to 2024 for AAPL and BTC-USD from open-source sources (Yahoo Finance).  
2. Clean and validate the dataset (missing values, duplicates, date range checks).  
3. Engineer interpretable technical indicators (SMA, EMA, RSI, MACD, Bollinger Bands, returns, volatility).  
4. Train Regression and baseline models (Linear regression, Ridge regression, Support vector Regression, Random Forest and XGBoost).  
5. Train sequence model (LSTM) and a hybrid RF→LSTM pipeline.  
6. Evaluate models using MAE, RMSE, MAPE and Directional Accuracy.  
7. Produce diagnostic plots and a results summary table.



## 1. Environment Setup and Reproducibility

This section defines the computational environment used to develop and evaluate all models in this project. Ensuring a stable and reproducible environment is essential for research, particularly when working with machine learning and deep learning frameworks.

The notebook is designed to run in a controlled Python environment with consistent library versions. This helps to minimise variability in results caused by software updates or dependency conflicts.

Key reproducibility measures include:
- Use of a fixed Python version and well-established libraries
- Explicit imports of all required packages
- Setting random seeds where applicable to reduce stochastic variation
- Using a fixed historical date range for all experiments

## Python Packages and Libraries

The following Python libraries were used throughout the project. 

| Package | Purpose in the Project |
|-------|------------------------|
| **numpy** | Numerical computing and efficient array operations |
| **pandas** | Data manipulation, time-series handling and preprocessing |
| **matplotlib** | Visualisation of price trends, predictions and diagnostics |
| **scikit-learn** | Classical machine learning models, preprocessing and evaluation metrics |
| **yfinance** | Retrieval of historical stock and cryptocurrency data from Yahoo Finance |
| **tensorflow** | Implementation and training of the LSTM deep learning model |
| **seaborn**  | Enhanced statistical visualisations |


### 1.1 Project Overview

This project investigates the application of hybrid machine learning models for predicting stock and cryptocurrency prices. Specifically, it focuses on Apple Inc.(AAPL) as a representative stock and Bitcoin (BTC-USD) as a representative cryptocurrency.

The primary objective is to design, implement and evaluate a hybrid modelling framework that combines tree-based machine learning models with deep learning techniques to capture both nonlinear feature relationships and temporal dependencies in financial time-series data.


### 1.2 Imports and configuration

This section imports all required Python libraries and dependencies used throughout
the notebook. The environment is configured to ensure reproducibility and
compatibility with the implemented machine learning and deep learning models.


In [ ]:
# ==============================
#        Imports
# ==============================

import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Yahoo Finance data
import yfinance as yf

# xgboost essentials
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler

# TensorFlow essentials
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# scikit-learn essentials
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline


In [ ]:
# ------------------------------
# Reproducibility
# ------------------------------
SEED = 42
np.random.seed(SEED)

# TensorFlow (for LSTM)
TF_AVAILABLE = True
try:
    tf.random.set_seed(SEED)
except Exception as e:
    TF_AVAILABLE = False
    print("[WARN] TensorFlow not available. LSTM sections will be skipped.", str(e)[:160])

# XGBoost baseline
XGB_AVAILABLE = True
try:
    from xgboost import XGBRegressor
except Exception as e:
    XGB_AVAILABLE = False
    print("[WARN] XGBoost not available. XGB baseline will be skipped.", str(e)[:160])


# ------------------------------
# Project folders
# ------------------------------
DATA_DIR = "data"
FIG_DIR = "figures"
OUT_DIR = "outputs"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

# ------------------------------
# Project settings
# ------------------------------
START_DATE = "2018-01-01"
END_DATE = "2024-12-31"  

STOCK_TICKER = "AAPL"
CRYPTO_TICKER = "BTC-USD"

LOOKBACK_WINDOW = 30     # LSTM input sequence length
TEST_SIZE = 0.20         # 80/20 holdout split (chronological)
SAVE_FIGS = True         # save figures to figures/ folder

print("Environment ready.")
print("TensorFlow available:", TF_AVAILABLE)
print("XGBoost available:", XGB_AVAILABLE)


### Helper function to save plots and tables

In [ ]:
def savefig(filename: str):
    """Save current matplotlib figure to the figures folder (300 dpi)."""
    if SAVE_FIGS:
        path = os.path.join(FIG_DIR, filename)
        plt.savefig(path, dpi=300, bbox_inches="tight")
        print(f"[Saved figure] {path}")


## 2. Data collection and initial validation

This stage focuses on collecting reliable historical price data required for model development.
Daily price data is retrieved for one stock market asset and one cryptocurrency asset:

- **Stock:** Apple Inc. (AAPL)
- **Cryptocurrency:** Bitcoin (BTC-USD)

The data is sourced from *Yahoo Finance*, accessed programmatically using the `yfinance` Python library.
Yahoo Finance provides adjusted daily OHLCV (Open, High, Low, Close, Volume) data.

To support transparency and reproducibility:
- Raw datasets are stored locally as CSV files.
- Basic data quality checks are performed immediately after download.
- The date range is fixed to maintain consistency across all experiments.

The goal is to validate that the raw data is complete, consistent and suitable for further analysis.



### 2.1 Data Collection

Historical price data for AAPL (stock) and BTC-USD (cryptocurrency) are collected using the Yahoo Finance API. The dataset spans multiple years from 2018 to 2024 to capture different market conditions, including stable, volatile and recovery periods.


In [ ]:
# ==================================
# 2.1 Download historical OHLCV data
# ==================================

def download_yahoo_data(ticker: str, start: str, end: str) -> pd.DataFrame:
    """
    Downloads daily OHLCV data from Yahoo Finance.
    Returns a cleaned DataFrame with a Date column.
    """
    df = yf.download(ticker, start=start, end=end, progress=False)

    if df.empty:
        raise ValueError(f"No data returned for ticker: {ticker}")

    df = df.reset_index()
    df["Ticker"] = ticker
    return df


# Download stock and crypto data
stock_df = download_yahoo_data(STOCK_TICKER, START_DATE, END_DATE)
crypto_df = download_yahoo_data(CRYPTO_TICKER, START_DATE, END_DATE)

# Save raw datasets for reproducibility
stock_df.to_csv(os.path.join(DATA_DIR, f"{STOCK_TICKER}_raw.csv"), index=False)
crypto_df.to_csv(os.path.join(DATA_DIR, f"{CRYPTO_TICKER}_raw.csv"), index=False)

print("Stock data shape:", stock_df.shape)
print("Crypto data shape:", crypto_df.shape)


### 2.2 Data Quality Assessment

This section performs an initial inspection of the raw datasets to assess dataquality. Checks include dataset dimensions, missing values, duplicate records and date ranges to ensure the data is suitable for further analysis.


In [ ]:
# =======================
# 2.2 Data quality 
# =======================

def data_quality_report(df: pd.DataFrame, name: str):
    """
    Prints a simple data quality report for a dataset.
    """
    print("\n" + "=" * 70)
    print(f"DATA QUALITY REPORT: {name}")
    print("=" * 70)
    print("Number of rows:", len(df))
    print("Number of columns:", df.shape[1])
    print("\nMissing values per column:")
    print(df.isnull().sum())
    print("\nDuplicate rows:", df.duplicated().sum())

    if "Date" in df.columns:
        print("\nDate range:")
        print("Start:", df["Date"].min())
        print("End:", df["Date"].max())
    print("=" * 70)


data_quality_report(stock_df, "AAPL (Stock)")
data_quality_report(crypto_df, "BTC-USD (Cryptocurrency)")


### 2.3 Exploratory Price Trend Visualization

This section visualizes the historical closing price trends for both AAPL and BTC-USD. The purpose is to gain an initial understanding of price behaviour, volatility patterns and long-term trends in stock and cryptocurrency markets.


In [ ]:
# ==================================
# 2.3 Initial price trend visualisation
# ==================================

plt.figure(figsize=(10, 4))
plt.plot(stock_df["Date"], stock_df["Close"], label="AAPL (Stock)")
plt.plot(crypto_df["Date"], crypto_df["Close"], label="BTC-USD (Crypto)")
plt.title("Closing Price Trends (2018–2024)")
plt.xlabel("Date")
plt.ylabel("Price (USD)")
plt.legend()
plt.tight_layout()
savefig("fig_raw_price_trends.png")
plt.show()


## 3. Data alignment and pre-processing

Financial time-series data must be aligned before any modelling can take place.
Stock markets operate only on business days, while cryptocurrency markets trade continuously.
As a result, stock and cryptocurrency datasets differ in both length and calendar structure.

To ensure consistency:
- Both datasets are aligned using a shared date index.
- Missing dates resulting from non-trading days are handled carefully.
- Forward-filling is applied to preserve the most recent available market information.
- The aligned dataset is converted into a wide format, enabling unified feature engineering.

This step ensures that all models are trained on a consistent temporal representation,
which is critical for valid comparison between stock and cryptocurrency predictions.


### 3.1 Date Alignment

Stock and cryptocurrency markets operate on different trading schedules. This section aligns the datasets by date to ensure consistency when analysing
cross-market relationships.


In [ ]:
# ==================================
# 3.1 Date formatting and sorting
# ==================================

stock_df["Date"] = pd.to_datetime(stock_df["Date"])
crypto_df["Date"] = pd.to_datetime(crypto_df["Date"])

stock_df = stock_df.sort_values("Date")
crypto_df = crypto_df.sort_values("Date")

print("Stock date range:", stock_df["Date"].min(), "to", stock_df["Date"].max())
print("Crypto date range:", crypto_df["Date"].min(), "to", crypto_df["Date"].max())


### 3.2 Combined Dataset Construction

This section merges the aligned stock and cryptocurrency datasets into a single time-indexed dataset. The combined dataset enables joint feature engineering and cross-market analysis.


In [ ]:
# ==================================
# 3.2 Align stock and crypto data
# ==================================

def align_time_series(stock_df: pd.DataFrame, crypto_df: pd.DataFrame) -> pd.DataFrame:
    """
    Aligns stock and cryptocurrency data on a common date index
    and returns a wide-format DataFrame.
    """

    stock_series = stock_df.set_index("Date")[["Close", "Volume"]]
    crypto_series = crypto_df.set_index("Date")[["Close", "Volume"]]

    stock_series.columns = ["Close_AAPL", "Volume_AAPL"]
    crypto_series.columns = ["Close_BTC", "Volume_BTC"]

    # Outer join to include all dates
    combined = pd.concat([stock_series, crypto_series], axis=1)

    # Forward-fill missing values caused by non-trading days
    combined = combined.ffill()
    return combined

aligned_df = align_time_series(stock_df, crypto_df)

print("Aligned dataset shape:", aligned_df.shape)
aligned_df.head()


### 3.3 Data Quality Check After Alignment

Following dataset alignment, additional data quality checks are performed to identify missing values or inconsistencies introduced during the merging process.


In [ ]:
# ==================================
# 3.3 Post-alignment data quality
# ==================================

data_quality_report(aligned_df.reset_index(), "Aligned stock-crypto dataset")


### 3.4 Aligned Price Visualization

This section visualizes the aligned closing prices for AAPL and BTC-USD to confirm that the datasets are synchronised correctly after preprocessing.


In [ ]:
# ==================================
# 3.4 Visual validation of alignment
# ==================================

plt.figure(figsize=(10, 4))
plt.plot(aligned_df.index, aligned_df["Close_AAPL"], label="AAPL (Aligned)")
plt.plot(aligned_df.index, aligned_df["Close_BTC"], label="BTC-USD (Aligned)")
plt.title("Aligned Closing Prices (Post Pre-processing)")
plt.xlabel("Date")
plt.ylabel("Price (USD)")
plt.legend()
plt.tight_layout()
savefig("aligned_price_trends.png")
plt.show()


### 3.5 Feature Correlation Analysis

A correlation heatmap is generated to examine relationships between stock and cryptocurrency features. This analysis provides insight into potential
interdependencies across markets.


In [ ]:
# ==================================
# 3.5 Remove initial non-trading day
# ==================================

aligned_df = aligned_df.dropna()

data_quality_report(aligned_df.reset_index(), "Aligned dataset (post-cleaning)")


## 4. Feature engineering

Raw price data alone is insufficient for effective financial prediction.Machine learning models benefit from engineered features that capture:

- Market trends
- Momentum
- Volatility
- Short-term and medium-term price behaviour

In this section, a set of widely used and technical indicators are constructed for both assets.

The selected features are:
- Simple Moving Average (SMA)
- Exponential Moving Average (EMA)
- Daily returns
- Rolling volatility
- Relative Strength Index (RSI)
- Bollinger Bands
- MACD (trend momentum indicator)

All features are computed using only historical data, avoiding data leakage.


### 4.1 Feature Engineering

This section generates technical indicators for both AAPL and BTC-USD, including moving averages, volatility measures, momentum indicators and trend-based features. These engineered features enhance the models ability to capture market dynamics.


In [ ]:
# ==================================
# 4.1 Technical indicator functions
# ==================================

def add_sma(df, col, window):
    df[f"{col}_SMA_{window}"] = df[col].rolling(window).mean()
    return df

def add_ema(df, col, span):
    df[f"{col}_EMA_{span}"] = df[col].ewm(span=span, adjust=False).mean()
    return df

def add_returns(df, col):
    df[f"{col}_RET"] = df[col].pct_change()
    return df

def add_volatility(df, col, window=10):
    df[f"{col}_VOL_{window}"] = df[col].pct_change().rolling(window).std()
    return df

def add_rsi(df, col, window=14):
    delta = df[col].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()
    rs = avg_gain / (avg_loss + 1e-9)
    df[f"{col}_RSI_{window}"] = 100 - (100 / (1 + rs))
    return df

def add_bollinger(df, col, window=20, num_std=2):
    mid = df[col].rolling(window).mean()
    std = df[col].rolling(window).std()
    df[f"{col}_BB_MID"] = mid
    df[f"{col}_BB_UP"] = mid + num_std * std
    df[f"{col}_BB_LOW"] = mid - num_std * std
    return df

def add_macd(df, col):
    ema_fast = df[col].ewm(span=12, adjust=False).mean()
    ema_slow = df[col].ewm(span=26, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal = macd.ewm(span=9, adjust=False).mean()
    df[f"{col}_MACD"] = macd
    df[f"{col}_MACD_SIGNAL"] = signal
    df[f"{col}_MACD_HIST"] = macd - signal
    return df


### 4.2 Validation of Engineered Features

After feature engineering, data quality checks are repeated to ensure that all newly created features are valid and free from missing values or inconsistencies.


In [ ]:
# ==================================
# 4.2 Build feature dataset
# ==================================

features_df = aligned_df.copy()

targets = ["Close_AAPL", "Close_BTC"]

for t in targets:
    features_df = add_sma(features_df, t, 5)
    features_df = add_sma(features_df, t, 10)
    features_df = add_ema(features_df, t, 10)
    features_df = add_returns(features_df, t)
    features_df = add_volatility(features_df, t, 10)
    features_df = add_rsi(features_df, t, 14)
    features_df = add_bollinger(features_df, t)
    features_df = add_macd(features_df, t)

# Remove rows created by rolling calculations
features_df = features_df.dropna()

data_quality_report(features_df.reset_index(), "Feature-engineered dataset")
features_df.head()


In [ ]:
# ==================================
# 4.3 Feature correlation analysis
# ==================================
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(9, 8))

sns.heatmap(
    features_df.corr(),
    cmap="coolwarm",
    center=0,
    linewidths=0.3,
    cbar_kws={"label": "Correlation"}
)

plt.title("Feature Correlation Heatmap", fontsize=9)
plt.tight_layout()
savefig("feature_correlation.png")
plt.show()


## 5. Supervised learning setup

After feature engineering, the problem must be formally defined as a supervised machine learning task.

In this project, the objective is to predict the *next-day closing price* based on historical price behaviour and technical indicators.
Although the model outputs a numeric price, the evaluation focuses on *trend prediction*, i.e., whether the model correctly predicts
the direction of price movement (upward or downward).

Key design decisions at this stage include:
- Defining the prediction target using a one-day forecast horizon
- Separating features from target variables
- Performing a *chronological train–test split* to prevent look-ahead bias
- Establishing evaluation metrics appropriate for financial time-series data

This formulation ensures that all models are trained and tested under realistic market conditions.


### 5.1 Target Variable Creation

In this section, the target variables are created for both stock and cryptocurrency datasets. The target represents the next-day closing price, enabling the models to learn one-step-ahead price prediction.


In [ ]:
# ==================================
# 5.1 Target variable construction
# ==================================

# Create next-day prediction targets
features_df["Target_AAPL"] = features_df["Close_AAPL"].shift(-1)
features_df["Target_BTC"] = features_df["Close_BTC"].shift(-1)

# Remove last row (no future target available)
features_df = features_df.dropna()

print("Dataset shape after target creation:", features_df.shape)
features_df[["Close_AAPL", "Target_AAPL", "Close_BTC", "Target_BTC"]].head()


In [ ]:
# ==================================
# 5.1.5 Direction Target Creation (NEW)
# ==================================

# Binary classification: 1 = price goes up, 0 = price goes down
features_df["Direction_AAPL"] = (
    features_df["Target_AAPL"] > features_df["Close_AAPL"]
).astype(int)

features_df["Direction_BTC"] = (
    features_df["Target_BTC"] > features_df["Close_BTC"]
).astype(int)

# Check class balance
print("AAPL Direction Distribution:")
print(features_df["Direction_AAPL"].value_counts())
print(f"Percentage going up: {features_df['Direction_AAPL'].mean():.2%}")

print("\nBTC Direction Distribution:")
print(features_df["Direction_BTC"].value_counts())
print(f"Percentage going up: {features_df['Direction_BTC'].mean():.2%}")

# Visualize direction distribution
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

features_df["Direction_AAPL"].value_counts().plot(
    kind='bar', ax=axes[0], color=['red', 'green']
)
axes[0].set_title("AAPL Direction Distribution")
axes[0].set_xlabel("Direction (0=Down, 1=Up)")
axes[0].set_ylabel("Count")
axes[0].set_xticklabels(['Down', 'Up'], rotation=0)

features_df["Direction_BTC"].value_counts().plot(
    kind='bar', ax=axes[1], color=['red', 'green']
)
axes[1].set_title("BTC Direction Distribution")
axes[1].set_xlabel("Direction (0=Down, 1=Up)")
axes[1].set_ylabel("Count")
axes[1].set_xticklabels(['Down', 'Up'], rotation=0)

plt.tight_layout()
savefig("direction_distribution.png")
plt.show()

### 5.2 Feature and Target Separation

This section separates the dataset into input features and target variables.The features are used as inputs to the machine learning models, while the target variables represent the values to be predicted.


In [ ]:
# ==================================
# 5.2 Feature and target separation 
# ==================================

# Define which columns to EXCLUDE from features
EXCLUDE_COLUMNS = [
    "Target_AAPL", 
    "Target_BTC",
    "Direction_AAPL",   
    "Direction_BTC",     
    "Close_AAPL",     
    "Close_BTC",       
]

FEATURE_COLUMNS = [
    col for col in features_df.columns
    if col not in EXCLUDE_COLUMNS
]

print("Excluded columns:", EXCLUDE_COLUMNS)
print("Number of features:", len(FEATURE_COLUMNS))
print("\nFeatures used:")
for i, col in enumerate(FEATURE_COLUMNS, 1):
    print(f"{i}. {col}")

X = features_df[FEATURE_COLUMNS]
y_stock = features_df["Target_AAPL"]
y_crypto = features_df["Target_BTC"]
y_stock_direction = features_df["Direction_AAPL"]
y_crypto_direction = features_df["Direction_BTC"]

print(f"\nX shape: {X.shape}")
print(f"y_stock shape: {y_stock.shape}")
print(f"y_crypto shape: {y_crypto.shape}")
print("Number of features:", X.shape[1])

### 5.3 Train and Test Split

The dataset is split into training and testing sets using a time-based approach.This ensures that future data is not used to predict past values and prevents data leakage.


In [ ]:
# ==================================
# 5.3 Time-aware train-test split 
# ==================================

def time_series_split(X, y, y_direction, test_size=0.2):
    """
    Splits data chronologically into training and testing sets.
    Now also splits direction targets.
    """
    split_idx = int(len(X) * (1 - test_size))
    X_train = X.iloc[:split_idx]
    X_test = X.iloc[split_idx:]
    y_train = y.iloc[:split_idx]
    y_test = y.iloc[split_idx:]
    y_dir_train = y_direction.iloc[:split_idx]
    y_dir_test = y_direction.iloc[split_idx:]
    return X_train, X_test, y_train, y_test, y_dir_train, y_dir_test

# Stock split
(X_train_stock, X_test_stock, 
 y_train_stock, y_test_stock,
 y_train_stock_dir, y_test_stock_dir) = time_series_split(
    X, y_stock, y_stock_direction, TEST_SIZE
)

# Crypto split
(X_train_crypto, X_test_crypto, 
 y_train_crypto, y_test_crypto,
 y_train_crypto_dir, y_test_crypto_dir) = time_series_split(
    X, y_crypto, y_crypto_direction, TEST_SIZE
)

print("Stock train size:", X_train_stock.shape)
print("Stock test size:", X_test_stock.shape)
print(f"Train direction split: {y_train_stock_dir.mean():.2%} up")
print(f"Test direction split: {y_test_stock_dir.mean():.2%} up")

### 5.4 Training and Testing Date Validation

This section verifies the date ranges of the training and testing datasets. It ensures that the chronological order of the data is preserved.


In [ ]:
# ==================================
# 5.4 Evaluation metrics
# ==================================

def regression_metrics(y_true, y_pred):
    """
    Computes standard regression metrics.
    """
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)     
    rmse = np.sqrt(mse)                          
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-9))) * 100
    return mae, rmse, mape


def directional_accuracy(y_true, y_pred):
    """
    Computes directional accuracy (trend prediction accuracy).
    """
    true_dir = np.sign(y_true.diff().dropna())
    pred_dir = np.sign(pd.Series(y_pred, index=y_true.index).diff().dropna())
    return (true_dir == pred_dir).mean()


### 5.5 Final Dataset Preparation for Modeling

This section confirms that the dataset is fully prepared for model training. The features, targets and splits are validated and ready to be used in
machine learning and deep learning models.


In [ ]:
# ==================================
# 5.5 Baseline model training template
# ==================================

def train_random_forest(X_train, y_train, n_estimators=200):
    """
    Trains a Random Forest regressor.
    """
    rf = RandomForestRegressor(
        n_estimators=n_estimators,
        random_state=SEED,
        n_jobs=-1
    )
    rf.fit(X_train, y_train)
    return rf


In [ ]:
# Checking dates for training and testing
print("First date in training:", X_train_stock.index.min())
print("Last date in training:", X_train_stock.index.max())
print("First date in testing:", X_test_stock.index.min())
print("Last date in testing:", X_test_stock.index.max())


## 6 Classical Regression and Baseline Models

Classical regression approaches are implemented as baseline methods. These models establish a
lower-bound benchmark for predictive performance and help assess the degree of non-linearity and temporal complexity present in the data.

The following regression models are considered:
- Linear Regression
- Ridge Regression (L2-regularised linear regression)
- Support Vector Regression (SVR) with RBF kernel

Baseline models provide a point of comparison and help determine whether more complex architectures offer meaningful performance improvements.

The following supervised learning models are implemented:
- Random Forest Regressor
- XGBoost Regressor 

Both models are trained separately on stock and cryptocurrency datasets. Performance is evaluated using both regression metrics and directional accuracy, reflecting the dual objectives of price estimation and trend prediction.

All models are trained using the same chronological train-test split to ensure fair comparison with subsequent models.


### 6.1 Linear Regression – Stock Prediction (AAPL)

Linear Regression is implemented as a simple baseline model that assumes a linear relationship between input features and the target variable. Although
financial markets are inherently non-linear, this model provides a useful reference point for evaluating more complex approaches.


In [ ]:

# Linear Regression
lr_stock = LinearRegression()
lr_stock.fit(X_train_stock, y_train_stock)

y_pred_lr_stock = lr_stock.predict(X_test_stock)

mae_lr_stock, rmse_lr_stock, mape_lr_stock = regression_metrics(
    y_test_stock, y_pred_lr_stock
)
dir_acc_lr_stock = directional_accuracy(y_test_stock, y_pred_lr_stock)

print("Linear Regression – AAPL")
print(f"MAE  : {mae_lr_stock:.4f}")
print(f"RMSE : {rmse_lr_stock:.4f}")
print(f"MAPE : {mape_lr_stock:.2f}%")
print(f"Directional Accuracy: {dir_acc_lr_stock:.2%}")


### 6.2 Ridge Regression – Stock Prediction (AAPL)

Ridge Regression extends linear regression by introducing L2 regularisation,which helps mitigate multicollinearity among technical indicators and reduces model variance.


In [ ]:
# Ridge Regression

ridge_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=1.0, random_state=SEED))
])

ridge_pipe.fit(X_train_stock, y_train_stock)
y_pred_ridge_stock = ridge_pipe.predict(X_test_stock)

mae_ridge_stock, rmse_ridge_stock, mape_ridge_stock = regression_metrics(y_test_stock, y_pred_ridge_stock)
dir_acc_ridge_stock = directional_accuracy(y_test_stock, y_pred_ridge_stock)

print("Ridge Regression – AAPL")
print(f"MAE  : {mae_ridge_stock:.4f}")
print(f"RMSE : {rmse_ridge_stock:.4f}")
print(f"MAPE : {mape_ridge_stock:.2f}%")
print(f"Directional Accuracy: {dir_acc_ridge_stock:.2%}")


### 6.3 Support Vector Regression – Stock Prediction (AAPL)

Support Vector Regression (SVR) with an RBF kernel is employed to model non-linear relationships between technical indicators and stock prices.
SVR serves as a strong classical machine learning baseline.


In [ ]:

# SVR requires scaling
svr_x_scaler = StandardScaler()
svr_y_scaler = StandardScaler()

X_train_svr = svr_x_scaler.fit_transform(X_train_stock)
X_test_svr = svr_x_scaler.transform(X_test_stock)

y_train_svr = svr_y_scaler.fit_transform(
    y_train_stock.values.reshape(-1, 1)
).ravel()

svr_stock = SVR(kernel="rbf", C=100, gamma="scale", epsilon=0.1)
svr_stock.fit(X_train_svr, y_train_svr)

y_pred_svr_scaled = svr_stock.predict(X_test_svr)
y_pred_svr_stock = svr_y_scaler.inverse_transform(
    y_pred_svr_scaled.reshape(-1, 1)
).ravel()

mae_svr_stock, rmse_svr_stock, mape_svr_stock = regression_metrics(
    y_test_stock, y_pred_svr_stock
)
dir_acc_svr_stock = directional_accuracy(y_test_stock, y_pred_svr_stock)

print("SVR (RBF) – AAPL")
print(f"MAE  : {mae_svr_stock:.4f}")
print(f"RMSE : {rmse_svr_stock:.4f}")
print(f"MAPE : {mape_svr_stock:.2f}%")
print(f"Directional Accuracy: {dir_acc_svr_stock:.2%}")


### 6.4 Random Forest Baseline – Stock Prediction

A Random Forest regression model is trained as a baseline for predicting AAPL prices. This model captures nonlinear feature relationships without considering temporal dependencies.


In [ ]:
# ==================================
# Random Forest – Stock (AAPL)
# ==================================

rf_stock = train_random_forest(X_train_stock, y_train_stock)

y_pred_rf_stock = rf_stock.predict(X_test_stock)

mae_s, rmse_s, mape_s = regression_metrics(y_test_stock, y_pred_rf_stock)
dir_acc_s = directional_accuracy(y_test_stock, y_pred_rf_stock)

print("Random Forest – AAPL")
print(f"MAE  : {mae_s:.4f}")
print(f"RMSE : {rmse_s:.4f}")
print(f"MAPE : {mape_s:.2f}%")
print(f"Directional Accuracy: {dir_acc_s:.2%}")


### 6.5 Random Forest Baseline – Cryptocurrency Prediction

The Random Forest model is applied to BTC-USD data to establish a baseline performance for cryptocurrency price prediction.


In [ ]:
# ==================================
# Random Forest – Crypto (BTC)
# ==================================

rf_crypto = train_random_forest(X_train_crypto, y_train_crypto)

y_pred_rf_crypto = rf_crypto.predict(X_test_crypto)

mae_c, rmse_c, mape_c = regression_metrics(y_test_crypto, y_pred_rf_crypto)
dir_acc_c = directional_accuracy(y_test_crypto, y_pred_rf_crypto)

print("Random Forest – BTC-USD")
print(f"MAE  : {mae_c:.4f}")
print(f"RMSE : {rmse_c:.4f}")
print(f"MAPE : {mape_c:.2f}%")
print(f"Directional Accuracy: {dir_acc_c:.2%}")


### 6.6 XGBoost Model – Stock Prediction

An XGBoost regression model is trained for AAPL to compare gradient-boosted trees against Random Forest and deep learning approaches.


In [ ]:
# ==================================
# XGBoost baseline 
# ==================================

if XGB_AVAILABLE:
    xgb_stock = XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=SEED
    )

    xgb_stock.fit(X_train_stock, y_train_stock)
    y_pred_xgb_stock = xgb_stock.predict(X_test_stock)

    mae_x, rmse_x, mape_x = regression_metrics(y_test_stock, y_pred_xgb_stock)
    dir_acc_x = directional_accuracy(y_test_stock, y_pred_xgb_stock)

    print("XGBoost – AAPL")
    print(f"MAE  : {mae_x:.4f}")
    print(f"RMSE : {rmse_x:.4f}")
    print(f"MAPE : {mape_x:.2f}%")
    print(f"Directional Accuracy: {dir_acc_x:.2%}")


### 6.6 Feature Importance Analysis

Feature importance scores from tree-based models are visualized to identify the most influential predictors contributing to price prediction.


In [ ]:
# ==================================
#  Feature importance analysis
# ==================================

importances = pd.Series(
    rf_stock.feature_importances_,
    index=X_train_stock.columns
).sort_values(ascending=False)

plt.figure(figsize=(8, 5))
importances.head(5).plot(kind="barh")
plt.gca().invert_yaxis()
plt.title("Top 5 Feature Importances (Random Forest – AAPL)")
plt.tight_layout()
savefig("rf_feature_importance.png")
plt.show()


In [ ]:
# ==================================
# 6.8 Hyperparameter Optimization (NEW)
# ==================================

from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint, uniform

print("=" * 70)
print("HYPERPARAMETER OPTIMIZATION - RANDOM FOREST")
print("=" * 70)

# Define parameter distribution
param_distributions = {
    'n_estimators': randint(100, 500),
    'max_depth': [10, 20, 30, 40, None],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

# Use TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Create base model
rf_base = RandomForestRegressor(random_state=SEED, n_jobs=-1)

# Randomized search
print("\nSearching for optimal hyperparameters...")
print("This may take a few minutes...")

random_search = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=param_distributions,
    n_iter=20,  # Number of parameter settings to try
    cv=tscv,
    scoring='neg_mean_absolute_error',
    random_state=SEED,
    n_jobs=-1,
    verbose=1
)

# Fit the random search
random_search.fit(X_train_stock, y_train_stock)

print("\n✓ Optimization complete!")
print("\nBest parameters found:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nBest cross-validation MAE: {-random_search.best_score_:.4f}")

# Train final model with best parameters
rf_optimized = random_search.best_estimator_
y_pred_rf_optimized = rf_optimized.predict(X_test_stock)

# Evaluate
mae_rf_opt, rmse_rf_opt, mape_rf_opt = regression_metrics(
    y_test_stock, y_pred_rf_optimized
)
dir_acc_rf_opt = directional_accuracy(y_test_stock, y_pred_rf_optimized)

print("\nOptimized Random Forest Performance:")
print(f"  MAE:                   {mae_rf_opt:.4f}")
print(f"  RMSE:                  {rmse_rf_opt:.4f}")
print(f"  MAPE:                  {mape_rf_opt:.2f}%")
print(f"  Directional Accuracy:  {dir_acc_rf_opt:.2%}")

print("\nComparison with default parameters:")
print(f"  MAE improvement:       {(1 - mae_rf_opt/mae_rf_stock)*100:.2f}%")
print(f"  RMSE improvement:      {(1 - rmse_rf_opt/rmse_rf_stock)*100:.2f}%")
print(f"  Dir_Acc improvement:   {(dir_acc_rf_opt - dir_acc_rf_stock)*100:.2f} percentage points")



In [ ]:
# Visualize hyperparameter search results
cv_results = pd.DataFrame(random_search.cv_results_)
cv_results['mean_test_mae'] = -cv_results['mean_test_score']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Number of estimators vs MAE
axes[0, 0].scatter(
    cv_results['param_n_estimators'],
    cv_results['mean_test_mae'],
    alpha=0.6,
    s=100
)
axes[0, 0].set_xlabel('Number of Estimators')
axes[0, 0].set_ylabel('Cross-Validation MAE')
axes[0, 0].set_title('n_estimators vs Performance')
axes[0, 0].grid(alpha=0.3)

# Plot 2: Max depth vs MAE
max_depth_values = [x if x is not None else 50 for x in cv_results['param_max_depth']]
axes[0, 1].scatter(
    max_depth_values,
    cv_results['mean_test_mae'],
    alpha=0.6,
    s=100
)
axes[0, 1].set_xlabel('Max Depth (None → 50)')
axes[0, 1].set_ylabel('Cross-Validation MAE')
axes[0, 1].set_title('max_depth vs Performance')
axes[0, 1].grid(alpha=0.3)

# Plot 3: Min samples split vs MAE
axes[1, 0].scatter(
cv_results['param_min_samples_split'],
cv_results['mean_test_mae'],
alpha=0.6,
s=100
)
axes[1, 0].set_xlabel('Min Samples Split')
axes[1, 0].set_ylabel('Cross-Validation MAE')
axes[1, 0].set_title('min_samples_split vs Performance')
axes[1, 0].grid(alpha=0.3)
# Plot 4: Performance over iterations
axes[1, 1].plot(
range(len(cv_results)),
cv_results['mean_test_mae'],
marker='o',
linestyle='-',
alpha=0.7
)
axes[1, 1].axhline(
y=cv_results['mean_test_mae'].min(),
color='red',
linestyle='--',
label=f'Best MAE: {cv_results["mean_test_mae"].min():.4f}'
)
axes[1, 1].set_xlabel('Iteration')
axes[1, 1].set_ylabel('Cross-Validation MAE')
axes[1, 1].set_title('Optimization Progress')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)
plt.tight_layout()
savefig("hyperparameter_optimization.png")
plt.show()
print("=" * 70)

In [ ]:
# ==================================
# Random Forest Classifier for Direction 
# ==================================

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve
)
import seaborn as sns

print("=" * 70)
print("RANDOM FOREST CLASSIFIER - AAPL")
print("=" * 70)

# Train classifier
rf_classifier_stock = RandomForestClassifier(
    n_estimators=300,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    class_weight='balanced',  # Handle any class imbalance
    random_state=SEED,
    n_jobs=-1
)

rf_classifier_stock.fit(X_train_stock, y_train_stock_dir)
y_pred_dir_rf_stock = rf_classifier_stock.predict(X_test_stock)
y_pred_proba_rf_stock = rf_classifier_stock.predict_proba(X_test_stock)[:, 1]

# Calculate metrics
acc_rf_stock = accuracy_score(y_test_stock_dir, y_pred_dir_rf_stock)
prec_rf_stock = precision_score(y_test_stock_dir, y_pred_dir_rf_stock)
rec_rf_stock = recall_score(y_test_stock_dir, y_pred_dir_rf_stock)
f1_rf_stock = f1_score(y_test_stock_dir, y_pred_dir_rf_stock)
auc_rf_stock = roc_auc_score(y_test_stock_dir, y_pred_proba_rf_stock)

print(f"Accuracy:  {acc_rf_stock:.2%}")
print(f"Precision: {prec_rf_stock:.2%}")
print(f"Recall:    {rec_rf_stock:.2%}")
print(f"F1-Score:  {f1_rf_stock:.2%}")
print(f"ROC AUC:   {auc_rf_stock:.2%}")

print("\nClassification Report:")
print(classification_report(
    y_test_stock_dir, 
    y_pred_dir_rf_stock,
    target_names=['Down', 'Up']
))


In [ ]:
# Confusion Matrix
cm_rf_stock = confusion_matrix(y_test_stock_dir, y_pred_dir_rf_stock)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Plot 1: Confusion Matrix
sns.heatmap(
    cm_rf_stock, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    ax=axes[0],
    xticklabels=['Predicted Down', 'Predicted Up'],
    yticklabels=['Actual Down', 'Actual Up']
)
axes[0].set_title("Confusion Matrix - RF Classifier (AAPL)")
axes[0].set_ylabel("True Label")
axes[0].set_xlabel("Predicted Label")

# Plot 2: ROC Curve
fpr, tpr, _ = roc_curve(y_test_stock_dir, y_pred_proba_rf_stock)
axes[1].plot(fpr, tpr, label=f'RF (AUC = {auc_rf_stock:.3f})', linewidth=2)
axes[1].plot([0, 1], [0, 1], 'k--', label='Random Classifier')
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve - RF Classifier (AAPL)')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
savefig("rf_classifier_aapl.png")
plt.show()

# Directional accuracy from regression predictions (Random Forest regression)
dir_acc_rf_stock = np.mean(
    np.sign(y_test_stock.values[1:] - y_test_stock.values[:-1]) ==
    np.sign(y_pred_rf_stock[1:] - y_pred_rf_stock[:-1])
)

# Compare with regression directional accuracy
print("\n" + "=" * 70)
print("COMPARISON: Regression vs Classification")
print("=" * 70)
print(f"Regression Model (Directional Accuracy): {dir_acc_rf_stock:.2%}")
print(f"Classification Model (Accuracy):         {acc_rf_stock:.2%}")
print(f"Improvement: {(acc_rf_stock - dir_acc_rf_stock):.2%}")
print("=" * 70)

In [ ]:
# ==================================
# 6.10 Random Forest Classifier - BTC 
# ==================================

print("\n" + "=" * 70)
print("RANDOM FOREST CLASSIFIER - BTC-USD")
print("=" * 70)

rf_classifier_crypto = RandomForestClassifier(
    n_estimators=300,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    class_weight='balanced',
    random_state=SEED,
    n_jobs=-1
)

rf_classifier_crypto.fit(X_train_crypto, y_train_crypto_dir)
y_pred_dir_rf_crypto = rf_classifier_crypto.predict(X_test_crypto)
y_pred_proba_rf_crypto = rf_classifier_crypto.predict_proba(X_test_crypto)[:, 1]

acc_rf_crypto = accuracy_score(y_test_crypto_dir, y_pred_dir_rf_crypto)
prec_rf_crypto = precision_score(y_test_crypto_dir, y_pred_dir_rf_crypto)
rec_rf_crypto = recall_score(y_test_crypto_dir, y_pred_dir_rf_crypto)
f1_rf_crypto = f1_score(y_test_crypto_dir, y_pred_dir_rf_crypto)
auc_rf_crypto = roc_auc_score(y_test_crypto_dir, y_pred_proba_rf_crypto)

print(f"Accuracy:  {acc_rf_crypto:.2%}")
print(f"Precision: {prec_rf_crypto:.2%}")
print(f"Recall:    {rec_rf_crypto:.2%}")
print(f"F1-Score:  {f1_rf_crypto:.2%}")
print(f"ROC AUC:   {auc_rf_crypto:.2%}")

print("\nClassification Report:")
print(classification_report(
    y_test_crypto_dir, 
    y_pred_dir_rf_crypto,
    target_names=['Down', 'Up']
))

## 7. LSTM model development

Traditional machine learning models treat observations as independent samples and therefore struggle to capture temporal dependencies present in financial time-series data.

Long Short-Term Memory (LSTM) networks are a type of recurrent neural network designed to learn sequential patterns over time.They are particularly well-suited for financial forecasting tasks where historical price movements influence future behaviour.

In this section, LSTM models are developed to predict next-day prices for:
- Apple Inc. (AAPL)
- Bitcoin (BTC-USD)

Key steps include:
- Feature scaling
- Sequence construction using a fixed lookback window
- LSTM model architecture design
- Model training with validation
- Performance evaluation on unseen test data


### 7.1 Data Scaling for LSTM

This section applies feature scaling to ensure that input variables are within a similar numerical range, which is critical for stable LSTM training.


In [ ]:
# ==================================
# 7.1 Feature scaling for LSTM
# ==================================

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X_train_scaled = x_scaler.fit_transform(X_train_stock)
X_test_scaled  = x_scaler.transform(X_test_stock)

# y must be 2D for scaler
y_train_scaled = y_scaler.fit_transform(y_train_stock.values.reshape(-1, 1))
y_test_scaled  = y_scaler.transform(y_test_stock.values.reshape(-1, 1))

print("X_train_scaled:", X_train_scaled.shape)
print("y_train_scaled:", y_train_scaled.shape)


### 7.2 LSTM Sequence Preparation

Input data is converted into rolling time-window sequences to enable the LSTM model to learn temporal dependencies in the financial time series.


In [ ]:
# ==================================
# 7.2 Create LSTM sequences
# ==================================

def create_sequences_xy(X, y, lookback):
    X_seq, y_seq = [], []
    for i in range(lookback, len(X)):
        X_seq.append(X[i - lookback:i])
        y_seq.append(y[i])  # scaled target
    return np.array(X_seq), np.array(y_seq)

X_train_seq, y_train_seq = create_sequences_xy(X_train_scaled, y_train_scaled, LOOKBACK_WINDOW)
X_test_seq, y_test_seq   = create_sequences_xy(X_test_scaled,  y_test_scaled,  LOOKBACK_WINDOW)

print("Train seq:", X_train_seq.shape, y_train_seq.shape)
print("Test seq :", X_test_seq.shape, y_test_seq.shape)


### 7.3 LSTM Model Architecture

An LSTM neural network is defined to model sequential dependencies in stock price data. The architecture includes dropout regularisation to mitigate overfitting.


In [ ]:
# ==================================
# 7.3 LSTM model architecture
# ==================================

def build_lstm(input_shape):
    """
    Builds a LSTM regression model.
    """
    model = Sequential([
        LSTM(64, return_sequences=False, input_shape=input_shape),
        Dropout(0.2),
        Dense(1)
    ])

    model.compile(
        optimizer="adam",
        loss="mse"
    )
    return model

lstm_model = build_lstm(
    input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])
)

lstm_model.summary()


### 7.4 LSTM Model Training

The LSTM model is trained using the prepared sequences. Training and validation losses are monitored to assess convergence and generalisation performance.


In [ ]:
# ==================================
# 7.4 Train LSTM model
# ==================================

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    min_lr=1e-5
)

history = lstm_model.fit(
    X_train_seq,
    y_train_seq,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


### 7.5 LSTM Model Evaluation

The trained LSTM model is evaluated on the test dataset using multiple error metrics and directional accuracy to assess predictive performance.


In [ ]:
# ==================================
# 7.5 Evaluate LSTM performance
# ==================================

y_pred_scaled = lstm_model.predict(X_test_seq).flatten()

# Inverse transform back to price scale
y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
y_true = y_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

mae_l, rmse_l, mape_l = regression_metrics(pd.Series(y_true), y_pred)
dir_acc_l = directional_accuracy(pd.Series(y_true), y_pred)

print("LSTM – AAPL (fixed scaling)")
print(f"MAE  : {mae_l:.4f}")
print(f"RMSE : {rmse_l:.4f}")
print(f"MAPE : {mape_l:.2f}%")
print(f"Directional Accuracy: {dir_acc_l:.2%}")


In [ ]:
# ==================================
# LSTM Classifier for Direction 
# ==================================

print("=" * 70)
print("LSTM CLASSIFIER - AAPL")
print("=" * 70)

# Need to create sequences for direction labels
def create_sequences_classification(X, y_direction, lookback):
    """
    Creates sequences for classification task.
    y_direction should be binary labels (0 or 1).
    """
    X_seq, y_seq = [], []
    for i in range(lookback, len(X)):
        X_seq.append(X[i - lookback:i])
        y_seq.append(y_direction[i])  # Binary label
    return np.array(X_seq), np.array(y_seq)

# Convert direction labels to numpy arrays
y_train_dir_arr = y_train_stock_dir.values
y_test_dir_arr = y_test_stock_dir.values

# Create sequences (reuse scaled data from Section 7.1)
X_train_seq_clf, y_train_seq_clf = create_sequences_classification(
    X_train_scaled, y_train_dir_arr, LOOKBACK_WINDOW
)
X_test_seq_clf, y_test_seq_clf = create_sequences_classification(
    X_test_scaled, y_test_dir_arr, LOOKBACK_WINDOW
)

print(f"Train sequences: {X_train_seq_clf.shape}, {y_train_seq_clf.shape}")
print(f"Test sequences:  {X_test_seq_clf.shape}, {y_test_seq_clf.shape}")

# Build LSTM Classifier
def build_lstm_classifier(input_shape):
    """
    Builds LSTM binary classifier.
    """
    model = Sequential([
        LSTM(64, return_sequences=False, input_shape=input_shape),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')  # ← Sigmoid for binary classification
    ])
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',  # ← Binary crossentropy for classification
        metrics=['accuracy']
    )
    return model

lstm_classifier = build_lstm_classifier(
    input_shape=(X_train_seq_clf.shape[1], X_train_seq_clf.shape[2])
)
lstm_classifier.summary()

# Train LSTM Classifier
history_clf = lstm_classifier.fit(
    X_train_seq_clf,
    y_train_seq_clf,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


In [ ]:
# Evaluate
y_pred_proba_lstm = lstm_classifier.predict(X_test_seq_clf).flatten()
y_pred_dir_lstm = (y_pred_proba_lstm > 0.5).astype(int)

acc_lstm_clf = accuracy_score(y_test_seq_clf, y_pred_dir_lstm)
prec_lstm_clf = precision_score(y_test_seq_clf, y_pred_dir_lstm)
rec_lstm_clf = recall_score(y_test_seq_clf, y_pred_dir_lstm)
f1_lstm_clf = f1_score(y_test_seq_clf, y_pred_dir_lstm)
auc_lstm_clf = roc_auc_score(y_test_seq_clf, y_pred_proba_lstm)

print("\nLSTM Classifier Results:")
print(f"Accuracy:  {acc_lstm_clf:.2%}")
print(f"Precision: {prec_lstm_clf:.2%}")
print(f"Recall:    {rec_lstm_clf:.2%}")
print(f"F1-Score:  {f1_lstm_clf:.2%}")
print(f"ROC AUC:   {auc_lstm_clf:.2%}")

print("\nClassification Report:")
print(classification_report(
    y_test_seq_clf, 
    y_pred_dir_lstm,
    target_names=['Down', 'Up']
))

# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss
axes[0].plot(history_clf.history['loss'], label='Training Loss')
axes[0].plot(history_clf.history['val_loss'], label='Validation Loss')
axes[0].set_title('LSTM Classifier Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Binary Crossentropy')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Accuracy
axes[1].plot(history_clf.history['accuracy'], label='Training Accuracy')
axes[1].plot(history_clf.history['val_accuracy'], label='Validation Accuracy')
axes[1].set_title('LSTM Classifier Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
savefig("lstm_classifier_training.png")
plt.show()

In [ ]:
# Alias LSTM regression outputs for consistency in later sections
y_pred_lstm = y_pred
y_true_lstm = y_true


In [ ]:
# ==================================
#  LSTM training loss plot
# ==================================

plt.figure(figsize=(7, 4))
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("LSTM Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.legend()
plt.tight_layout()
savefig("lstm_training_loss.png")
plt.show()

## 8. Hybrid RF → LSTM model

A hybrid modelling strategy is used to combine the strengths of tree-based feature selection and sequence-based deep learning.

- Random Forest models are effective at identifying informative features using feature importance measures.
- LSTMs are effective at learning sequential patterns over time, but they may perform poorly if trained on noisy or redundant features.

This hybrid approach proceeds in two stages:

1. Train a Random Forest model on the training set.
2. Select the top N features based on feature importances.
3. Train an LSTM model using only these selected features.

This design improves interpretability and can enhance performance by reducing input dimensionality and limiting noise in the LSTM learning process.


### 8.1 Feature Selection Using Random Forest

Random Forest feature importance is used to select a reduced subset of influential features. This step aims to improve LSTM efficiency and reduce noise.


In [ ]:
# ==================================
# 8.1 Select top features using RF importance (AAPL)
# ==================================

def get_top_features_from_rf(rf_model, feature_names, top_n=5):
    """
    Returns top_n feature names based on Random Forest feature importances.
    """
    imp = pd.Series(rf_model.feature_importances_, index=feature_names)
    top_features = imp.sort_values(ascending=False).head(top_n).index.tolist()
    return top_features, imp.sort_values(ascending=False)

TOP_N = 5

top_features_aapl, full_importance_aapl = get_top_features_from_rf(
    rf_stock, X_train_stock.columns, top_n=TOP_N
)

print("Top features (AAPL):")
for i, f in enumerate(top_features_aapl, start=1):
    print(f"{i}. {f}")


### 8.2 Reduced Feature Dataset Preparation

The dataset is reconstructed using only the selected top features, preparing it for hybrid model training.


In [ ]:
# ==================================
# 8.2 Reduce dataset to top features + scale X and y
# ==================================

# Use only selected top features
X_train_top = X_train_stock[top_features_aapl]
X_test_top  = X_test_stock[top_features_aapl]

# Scale X and y (train only fit → test transform)
x_scaler_h = MinMaxScaler()
y_scaler_h = MinMaxScaler()

X_train_top_scaled = x_scaler_h.fit_transform(X_train_top)
X_test_top_scaled  = x_scaler_h.transform(X_test_top)

y_train_h_scaled = y_scaler_h.fit_transform(y_train_stock.values.reshape(-1, 1))
y_test_h_scaled  = y_scaler_h.transform(y_test_stock.values.reshape(-1, 1))

print("Reduced X_train shape:", X_train_top_scaled.shape)
print("Reduced X_test shape:", X_test_top_scaled.shape)


### 8.3 Hybrid Model Sequence Preparation

Time-series sequences are generated from the reduced feature set for training the hybrid RF→LSTM model.


In [ ]:
# ==================================
# 8.3 Create sequences for hybrid LSTM
# ==================================

X_train_h_seq, y_train_h_seq = create_sequences_xy(
    X_train_top_scaled, y_train_h_scaled, LOOKBACK_WINDOW
)

X_test_h_seq, y_test_h_seq = create_sequences_xy(
    X_test_top_scaled, y_test_h_scaled, LOOKBACK_WINDOW
)

print("Hybrid train sequences:", X_train_h_seq.shape, y_train_h_seq.shape)
print("Hybrid test sequences :", X_test_h_seq.shape, y_test_h_seq.shape)


### 8.4 Hybrid RF→LSTM Model Training

The hybrid model is trained using the selected features and LSTM architecture.Training behaviour is monitored through loss curves.


In [ ]:
# ==================================
# 8.4 Build + train hybrid LSTM model
# ==================================

hybrid_lstm = build_lstm(
    input_shape=(X_train_h_seq.shape[1], X_train_h_seq.shape[2])
)

hybrid_lstm.summary()

history_hybrid = hybrid_lstm.fit(
    X_train_h_seq,
    y_train_h_seq,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


### 8.5 Hybrid Model Evaluation – Stock

The hybrid RF→LSTM model is evaluated on AAPL test data and compared against baseline models to quantify performance improvements.


In [ ]:
# ==================================
# 8.5 Evaluate hybrid RF → LSTM model
# ==================================

y_pred_h_scaled = hybrid_lstm.predict(X_test_h_seq).flatten()

# Inverse scale predictions back to real prices
y_pred_h = y_scaler_h.inverse_transform(y_pred_h_scaled.reshape(-1, 1)).flatten()
y_true_h = y_scaler_h.inverse_transform(y_test_h_seq.reshape(-1, 1)).flatten()

mae_h, rmse_h, mape_h = regression_metrics(pd.Series(y_true_h), y_pred_h)
dir_acc_h = directional_accuracy(pd.Series(y_true_h), y_pred_h)

print("Hybrid RF → LSTM – AAPL")
print(f"MAE  : {mae_h:.4f}")
print(f"RMSE : {rmse_h:.4f}")
print(f"MAPE : {mape_h:.2f}%")
print(f"Directional Accuracy: {dir_acc_h:.2%}")


In [ ]:
# ==================================
# 8.6 Hybrid training loss curve
# ==================================

plt.figure(figsize=(7, 4))
plt.plot(history_hybrid.history["loss"], label="Training Loss")
plt.plot(history_hybrid.history["val_loss"], label="Validation Loss")
plt.title("Hybrid RF→LSTM Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.legend()
plt.tight_layout()
savefig("hybrid_lstm_loss_aapl.png")
plt.show()


## 9. Model evaluation and visual diagnostics

Quantitative metrics alone are insufficient to fully assess forecasting models.
Therefore, visual diagnostic tools are employed to evaluate prediction quality, error behaviour and model stability.

This section presents:
- Actual vs predicted price plots
- Residual diagnostics
- Error distribution analysis
- Comparative performance summary across models

These visualisations support interpretability and provide empirical evidence for the effectiveness of the proposed hybrid model.


### 9.1 Actual vs Predicted Price Comparison

This section visualizes the actual and predicted stock prices over time to assess how well the hybrid model captures market dynamics.


In [ ]:
# ==================================
# 9.1 Actual vs Predicted (Hybrid – AAPL)
# ==================================

plt.figure(figsize=(10, 4))
plt.plot(y_true_h, label="Actual Price", linewidth=2)
plt.plot(y_pred_h, label="Predicted Price", linestyle="--")
plt.title("AAPL: Actual vs Predicted Prices (Hybrid RF→LSTM)")
plt.xlabel("Time Index (Test Period)")
plt.ylabel("Price (USD)")
plt.legend()
plt.tight_layout()
savefig("actual_vs_predicted_aapl.png")
plt.show()


### 9.2 Scatter Plot of Actual vs Predicted Prices

A scatter plot is generated to examine the agreement between actual and predicted prices and identify potential systematic biases.


In [ ]:
# ==================================
# 9.2 Scatter plot (Hybrid – AAPL)
# ==================================

plt.figure(figsize=(7, 5))
plt.scatter(y_true_h, y_pred_h, alpha=0.6)
min_val = min(y_true_h.min(), y_pred_h.min())
max_val = max(y_true_h.max(), y_pred_h.max())
plt.plot([min_val, max_val], [min_val, max_val], 'k--', linewidth=1)

plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.title("Actual vs Predicted Scatter Plot (Hybrid RF→LSTM)")
plt.tight_layout()
savefig("scatter_aapl.png")
plt.show()


### 9.3 Residual Analysis

Residuals are analysed over time to evaluate prediction errors and detect systematic patterns or model bias.


In [ ]:
# ==================================
# 9.3 Residual analysis (Hybrid – AAPL)
# ==================================

residuals_h = y_true_h - y_pred_h

plt.figure(figsize=(10, 4))
plt.plot(residuals_h)
plt.axhline(0, color="black", linestyle="--")
plt.title("Residuals over Time (Hybrid RF→LSTM – AAPL)")
plt.xlabel("Time Index")
plt.ylabel("Prediction Error (USD)")
plt.tight_layout()
savefig("residuals_aapl.png")
plt.show()


### 9.4 Error Distribution Analysis

The distribution of prediction errors is examined to assess the stability and robustness of the hybrid model.


In [ ]:
# ==================================
# 9.4 Error distribution (Hybrid – AAPL)
# ==================================


plt.figure(figsize=(7, 4))
plt.hist(
    residuals_h,
    bins=35,
    color="steelblue",
    edgecolor="black",
    alpha=0.85
)

# Reference lines
plt.axvline(0, color="black", linestyle="--", linewidth=1, label="Zero Error")
plt.axvline(
    residuals_h.mean(),
    color="red",
    linestyle=":",
    linewidth=1.5,
    label="Mean Error"
)
plt.axvline(
    np.median(residuals_h),
    color="green",
    linestyle="-.",
    linewidth=1.5,
    label="Median Error"
)

plt.title("Error Distribution of Hybrid RF→LSTM Model (AAPL)")
plt.xlabel("Prediction Error (USD)")
plt.ylabel("Frequency")
plt.legend()
plt.tight_layout()
savefig("error_distribution_aapl.png")
plt.show()


### 9.5 Model Performance Comparison

This section compares all implemented models using quantitative evaluation metrics to identify the best-performing approach.


In [ ]:
# ==================================
# 9.5 Store evaluation metrics explicitly
# ==================================

# Random Forest – AAPL
mae_rf_stock, rmse_rf_stock, mape_rf_stock = regression_metrics(y_test_stock, y_pred_rf_stock)
dir_acc_rf_stock = directional_accuracy(y_test_stock, y_pred_rf_stock)

# XGBoost – AAPL
mae_xgb_stock, rmse_xgb_stock, mape_xgb_stock = regression_metrics(y_test_stock, y_pred_xgb_stock)
dir_acc_xgb_stock = directional_accuracy(y_test_stock, y_pred_xgb_stock)

# LSTM – AAPL
mae_l = mae_l
rmse_l = rmse_l
mape_l = mape_l
dir_acc_l = dir_acc_l

# Hybrid RF → LSTM – AAPL
mae_h = mae_h
rmse_h = rmse_h
mape_h = mape_h
dir_acc_h = dir_acc_h


In [ ]:
# ==================================
# 9.7 Trading Simulation
# ==================================

def backtest_trading_strategy(
    actual_prices, 
    predictions, 
    initial_capital=10000,
    transaction_cost=0.001,
    strategy_type='regression'
):
    """
    Simulates realistic trading based on model predictions.
    
    Parameters:
    -----------
    actual_prices : array-like
        Actual closing prices
    predictions : array-like
        Model predictions (prices for regression, probabilities for classification)
    initial_capital : float
        Starting capital in USD
    transaction_cost : float
        Transaction cost as percentage (0.001 = 0.1%)
    strategy_type : str
        'regression' or 'classification'
    
    Returns:
    --------
    dict : Trading results and metrics
    """
    capital = initial_capital
    shares = 0
    position_value = 0
    
    portfolio_values = []
    trades = []
    
    for i in range(len(actual_prices) - 1):
        current_price = actual_prices[i]
        next_price = actual_prices[i + 1]
        
        # Generate trading signal
        if strategy_type == 'regression':
            # Buy if predicting price increase
            signal = 'buy' if predictions[i + 1] > current_price else 'sell'
        else:  # classification
            # Buy if predicting up movement (probability > 0.5)
            signal = 'buy' if predictions[i] > 0.5 else 'sell'
        
        # Execute trade
        if signal == 'buy' and capital > 0 and shares == 0:
            # Buy shares
            cost_per_share = current_price * (1 + transaction_cost)
            shares = capital / cost_per_share
            capital = 0
            trades.append({
                'day': i,
                'action': 'BUY',
                'price': current_price,
                'shares': shares,
                'cost': cost_per_share * shares
            })
            
        elif signal == 'sell' and shares > 0:
            # Sell shares
            revenue_per_share = current_price * (1 - transaction_cost)
            capital = shares * revenue_per_share
            trades.append({
                'day': i,
                'action': 'SELL',
                'price': current_price,
                'shares': shares,
                'revenue': capital
            })
            shares = 0
        
        # Calculate portfolio value
        if shares > 0:
            position_value = shares * current_price
        else:
            position_value = capital
        
        portfolio_values.append(position_value)
    
    # Final liquidation
    if shares > 0:
        final_price = actual_prices[-1]
        capital = shares * final_price * (1 - transaction_cost)
        shares = 0
    
    final_value = capital
    portfolio_values.append(final_value)
    
    # Calculate metrics
    total_return = (final_value / initial_capital - 1) * 100
    
    # Buy and hold comparison
    buy_hold_shares = initial_capital / (actual_prices[0] * (1 + transaction_cost))
    buy_hold_final = buy_hold_shares * actual_prices[-1] * (1 - transaction_cost)
    buy_hold_return = (buy_hold_final / initial_capital - 1) * 100
    
    # Calculate Sharpe ratio
    portfolio_returns = np.diff(portfolio_values) / portfolio_values[:-1]
    sharpe_ratio = np.mean(portfolio_returns) / (np.std(portfolio_returns) + 1e-9) * np.sqrt(252)
    
    # Calculate maximum drawdown
    portfolio_values_arr = np.array(portfolio_values)
    running_max = np.maximum.accumulate(portfolio_values_arr)
    drawdown = (portfolio_values_arr - running_max) / running_max
    max_drawdown = np.min(drawdown) * 100
    
    return {
        'Initial Capital': initial_capital,
        'Final Value': final_value,
        'Total Return (%)': total_return,
        'Buy & Hold Return (%)': buy_hold_return,
        'Excess Return (%)': total_return - buy_hold_return,
        'Number of Trades': len(trades),
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown (%)': max_drawdown,
        'Portfolio Values': portfolio_values,
        'Trades': trades
    }



In [ ]:
# Test with Hybrid RF→LSTM model (regression)
print("=" * 70)
print("TRADING SIMULATION - HYBRID RF→LSTM (Regression)")
print("=" * 70)

# Need to align predictions with actual prices
# y_true_h has 477 values, y_pred_h has 477 values
trading_results_regression = backtest_trading_strategy(
    actual_prices=y_true_h,
    predictions=y_pred_h,
    initial_capital=10000,
    transaction_cost=0.001,
    strategy_type='regression'
)

print(f"Initial Capital:      ${trading_results_regression['Initial Capital']:,.2f}")
print(f"Final Value:          ${trading_results_regression['Final Value']:,.2f}")
print(f"Total Return:         {trading_results_regression['Total Return (%)']:.2f}%")
print(f"Buy & Hold Return:    {trading_results_regression['Buy & Hold Return (%)']:.2f}%")
print(f"Excess Return:        {trading_results_regression['Excess Return (%)']:.2f}%")
print(f"Number of Trades:     {trading_results_regression['Number of Trades']}")
print(f"Sharpe Ratio:         {trading_results_regression['Sharpe Ratio']:.2f}")
print(f"Max Drawdown:         {trading_results_regression['Max Drawdown (%)']:.2f}%")

# Test with RF Classifier
print("\n" + "=" * 70)
print("TRADING SIMULATION - RF CLASSIFIER")
print("=" * 70)

trading_results_classifier = backtest_trading_strategy(
    actual_prices=y_test_stock.values,
    predictions=y_pred_proba_rf_stock,
    initial_capital=10000,
    transaction_cost=0.001,
    strategy_type='classification'
)

print(f"Initial Capital:      ${trading_results_classifier['Initial Capital']:,.2f}")
print(f"Final Value:          ${trading_results_classifier['Final Value']:,.2f}")
print(f"Total Return:         {trading_results_classifier['Total Return (%)']:.2f}%")
print(f"Buy & Hold Return:    {trading_results_classifier['Buy & Hold Return (%)']:.2f}%")
print(f"Excess Return:        {trading_results_classifier['Excess Return (%)']:.2f}%")
print(f"Number of Trades:     {trading_results_classifier['Number of Trades']}")
print(f"Sharpe Ratio:         {trading_results_classifier['Sharpe Ratio']:.2f}")
print(f"Max Drawdown:         {trading_results_classifier['Max Drawdown (%)']:.2f}%")


In [ ]:
# Visualize portfolio growth
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Plot 1: Regression Strategy
axes[0].plot(
    trading_results_regression['Portfolio Values'], 
    label='Hybrid RF→LSTM Strategy',
    linewidth=2
)
axes[0].axhline(
    y=10000, 
    color='gray', 
    linestyle='--', 
    label='Initial Capital',
    alpha=0.7
)
# Buy & Hold line
buy_hold_values = 10000 * (y_true_h / y_true_h[0])
axes[0].plot(
    buy_hold_values, 
    label='Buy & Hold',
    linestyle='--',
    alpha=0.7
)
axes[0].set_title('Portfolio Value - Regression Strategy (AAPL)')
axes[0].set_xlabel('Trading Days')
axes[0].set_ylabel('Portfolio Value ($)')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot 2: Classification Strategy
axes[1].plot(
    trading_results_classifier['Portfolio Values'], 
    label='RF Classifier Strategy',
    linewidth=2,
    color='green'
)
axes[1].axhline(
    y=10000, 
    color='gray', 
    linestyle='--', 
    label='Initial Capital',
    alpha=0.7
)
# Buy & Hold line
buy_hold_values_clf = 10000 * (y_test_stock.values / y_test_stock.values[0])
axes[1].plot(
    buy_hold_values_clf, 
    label='Buy & Hold',
    linestyle='--',
    alpha=0.7
)
axes[1].set_title('Portfolio Value - Classification Strategy (AAPL)')
axes[1].set_xlabel('Trading Days')
axes[1].set_ylabel('Portfolio Value ($)')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
savefig("trading_simulation_comparison.png")
plt.show()

# Summary comparison
print("\n" + "=" * 70)
print("STRATEGY COMPARISON SUMMARY")
print("=" * 70)
print(f"{'Metric':<30} {'Regression':<15} {'Classification':<15}")
print("-" * 70)
print(f"{'Total Return (%)':<30} {trading_results_regression['Total Return (%)']:>14.2f} {trading_results_classifier['Total Return (%)']:>14.2f}")
print(f"{'Buy & Hold Return (%)':<30} {trading_results_regression['Buy & Hold Return (%)']:>14.2f} {trading_results_classifier['Buy & Hold Return (%)']:>14.2f}")
print(f"{'Excess Return (%)':<30} {trading_results_regression['Excess Return (%)']:>14.2f} {trading_results_classifier['Excess Return (%)']:>14.2f}")
print(f"{'Number of Trades':<30} {trading_results_regression['Number of Trades']:>14} {trading_results_classifier['Number of Trades']:>14}")
print(f"{'Sharpe Ratio':<30} {trading_results_regression['Sharpe Ratio']:>14.2f} {trading_results_classifier['Sharpe Ratio']:>14.2f}")
print(f"{'Max Drawdown (%)':<30} {trading_results_regression['Max Drawdown (%)']:>14.2f} {trading_results_classifier['Max Drawdown (%)']:>14.2f}")
print("=" * 70)

In [ ]:
# ==================================
# 9.6 Results comparison table (AAPL)
# ==================================

results_aapl = pd.DataFrame({
    "Model": [
        "Random Forest",
        "XGBoost",
        "LSTM",
        "Hybrid RF→LSTM"
    ],
    "MAE": [
        mae_rf_stock,
        mae_xgb_stock,
        mae_l,
        mae_h
    ],
    "RMSE": [
        rmse_rf_stock,
        rmse_xgb_stock,
        rmse_l,
        rmse_h
    ],
    "MAPE (%)": [
        mape_rf_stock,
        mape_xgb_stock,
        mape_l,
        mape_h
    ],
    "Directional Accuracy (%)": [
        dir_acc_rf_stock * 100,
        dir_acc_xgb_stock * 100,
        dir_acc_l * 100,
        dir_acc_h * 100
    ]
})

results_aapl


## 10. Hybrid RF → LSTM model for cryptocurrency prediction

To assess the generalisability of the proposed hybrid framework, the same methodology is applied to cryptocurrency price prediction using Bitcoin (BTC-USD).

Cryptocurrency markets differ from equity markets in terms of volatility, trading hours and investor behaviour. Therefore, evaluating the hybrid model
on BTC-USD provides an important robustness check.

The modelling pipeline remains unchanged to ensure a fair comparison:
- Random Forest is used for feature selection
- LSTM models temporal dependencies using the selected features
- Performance is evaluated on a chronologically held-out test set


### 10.1 Hybrid RF→LSTM Training for Cryptocurrency

This section visualises the training and validation loss curves of the hybrid Random Forest → LSTM model for BTC-USD. Monitoring these curves helps assess model convergence, stability and potential overfitting during the training process on cryptocurrency price data.


In [ ]:
# ==================================
# 10.1 Random Forest baseline (BTC)
# ==================================

rf_crypto = RandomForestRegressor(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)

rf_crypto.fit(X_train_crypto, y_train_crypto)

y_pred_rf_crypto = rf_crypto.predict(X_test_crypto)

mae_rf_crypto, rmse_rf_crypto, mape_rf_crypto = regression_metrics(y_test_crypto, y_pred_rf_crypto)
dir_acc_rf_crypto = directional_accuracy(y_test_crypto, y_pred_rf_crypto)

print("Random Forest – BTC-USD")
print(f"MAE  : {mae_rf_crypto:.4f}")
print(f"RMSE : {rmse_rf_crypto:.4f}")
print(f"MAPE : {mape_rf_crypto:.2f}%")
print(f"Directional Accuracy: {dir_acc_rf_crypto:.2%}")


### 10.2 Random Forest Baseline Evaluation – BTC-USD

A Random Forest regression model is evaluated as a baseline for cryptocurrency price prediction. The performance metrics obtained here serve as a reference point for assessing the effectiveness of the hybrid modelling approach.


In [ ]:
# ==================================
# 10.2 RF feature selection (BTC)
# ==================================

TOP_N = 10

top_features_btc, btc_importance = get_top_features_from_rf(rf_crypto, X_train_crypto.columns, top_n=TOP_N)

print("Top features (BTC-USD):")
for i, f in enumerate(top_features_btc, start=1):
    print(f"{i}. {f}")


### 10.3 Feature Importance-Based Feature Selection (BTC-USD)

This section identifies the most influential features for BTC-USD prediction using Random Forest feature importance scores. Selected features include both cryptocurrency-specific indicators and cross-market stock indicators, supporting the hypothesis of inter-market relationships.


In [ ]:
# ==================================
# 10.3 Reduce features and scale X & y (BTC)
# ==================================

X_train_btc_top = X_train_crypto[top_features_btc]
X_test_btc_top  = X_test_crypto[top_features_btc]

x_scaler_btc = MinMaxScaler()
y_scaler_btc = MinMaxScaler()

X_train_btc_scaled = x_scaler_btc.fit_transform(X_train_btc_top)
X_test_btc_scaled  = x_scaler_btc.transform(X_test_btc_top)

y_train_btc_scaled = y_scaler_btc.fit_transform(
    y_train_crypto.values.reshape(-1, 1)
)
y_test_btc_scaled = y_scaler_btc.transform(
    y_test_crypto.values.reshape(-1, 1)
)

print("Reduced BTC X_train shape:", X_train_btc_scaled.shape)
print("Reduced BTC X_test shape :", X_test_btc_scaled.shape)


### 10.4 Reduced Feature Dataset Construction for BTC-USD

Based on the selected important features, a reduced feature dataset is created for BTC-USD. This dimensionality reduction aims to improve LSTM efficiency and reduce noise while preserving predictive information.


In [ ]:
# ==================================
# 10.4 Create sequences for hybrid BTC model
# ==================================

X_train_btc_seq, y_train_btc_seq = create_sequences_xy(
    X_train_btc_scaled, y_train_btc_scaled, LOOKBACK_WINDOW
)

X_test_btc_seq, y_test_btc_seq = create_sequences_xy(
    X_test_btc_scaled, y_test_btc_scaled, LOOKBACK_WINDOW
)

print("BTC train sequences:", X_train_btc_seq.shape, y_train_btc_seq.shape)
print("BTC test sequences :", X_test_btc_seq.shape, y_test_btc_seq.shape)


### 10.5 Hybrid RF→LSTM Model Training – Cryptocurrency

The hybrid RF→LSTM model is trained on the reduced BTC-USD feature set using time-series sequences. Learning rate scheduling and validation monitoring are applied to enhance training stability and generalisation performance.


In [ ]:
# ==================================
# 10.5 Train hybrid RF → LSTM (BTC)
# ==================================

hybrid_lstm_btc = build_lstm(
    input_shape=(X_train_btc_seq.shape[1], X_train_btc_seq.shape[2])
)

hybrid_lstm_btc.summary()

history_hybrid_btc = hybrid_lstm_btc.fit(
    X_train_btc_seq,
    y_train_btc_seq,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


### 10.6 Hybrid RF→LSTM Model Evaluation – BTC-USD

The trained hybrid model is evaluated on unseen BTC-USD test data using multiple error metrics and directional accuracy. The results are compared against baseline models to assess the hybrid model’s effectiveness in handling highly volatile cryptocurrency markets.


In [ ]:
# ==================================
# 10.6 Evaluate hybrid BTC model
# ==================================

y_pred_btc_scaled = hybrid_lstm_btc.predict(X_test_btc_seq).flatten()

y_pred_btc = y_scaler_btc.inverse_transform(
    y_pred_btc_scaled.reshape(-1, 1)
).flatten()

y_true_btc = y_scaler_btc.inverse_transform(
    y_test_btc_seq.reshape(-1, 1)
).flatten()

mae_h_btc, rmse_h_btc, mape_h_btc = regression_metrics(
    pd.Series(y_true_btc), y_pred_btc
)
dir_acc_h_btc = directional_accuracy(
    pd.Series(y_true_btc), y_pred_btc
)

print("Hybrid RF → LSTM – BTC-USD")
print(f"MAE  : {mae_h_btc:.4f}")
print(f"RMSE : {rmse_h_btc:.4f}")
print(f"MAPE : {mape_h_btc:.2f}%")
print(f"Directional Accuracy: {dir_acc_h_btc:.2%}")


### 10.7 Training and Validation Loss Visualisation – BTC-USD

This section visualises the training and validation loss curves of the hybrid RF→LSTM model for BTC-USD. The plot provides insight into the model’s learning behaviour, convergence stability and generalisation performance when applied to highly volatile cryptocurrency price data.


In [ ]:
# ==================================
# 10.7 Hybrid BTC training loss
# ==================================

plt.figure(figsize=(7, 4))
plt.plot(history_hybrid_btc.history["loss"], label="Training Loss")
plt.plot(history_hybrid_btc.history["val_loss"], label="Validation Loss")
plt.title("Hybrid RF→LSTM Training Loss (BTC-USD)")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.legend()
plt.tight_layout()
savefig("training_loss_btc.png")
plt.show()


## 11. Cross-asset performance comparison

This section compares the predictive performance of all models across two distinct financial markets: equities (AAPL) and cryptocurrencies (BTC-USD).

This section extends the cross-asset results tables by including classical regression baselines and baseline models.These models provide a lower-bound benchmark and strengthen the evaluation framework for hybrid modelling methodology.

By analysing error metrics and directional accuracy across asset classes, this comparison evaluates the robustness, generalisability and limitations
of the proposed hybrid RF→LSTM framework.

Such cross-market evaluation is essential for understanding how marketstructure, volatility and trading behaviour influence predictive performance.


### 11.1 Cross-Asset Performance Comparison

Model performance is compared across stock and cryptocurrency markets to analyse differences in predictive behaviour and robustness.



In [ ]:
# ==================================
# AAPL model table
# ==================================

results_aapl_expanded = pd.DataFrame({
    "Asset": ["AAPL"] * 7,
    "Model": [
        "Linear Regression",
        "Ridge Regression",
        "SVR (RBF)",
        "Random Forest",
        "XGBoost",
        "LSTM",
        "Hybrid RF→LSTM"
    ],
    "MAE": [
        mae_lr_stock,
        mae_ridge_stock,
        mae_svr_stock,
        mae_rf_stock,
        mae_xgb_stock,
        mae_l,
        mae_h
    ],
    "RMSE": [
        rmse_lr_stock,
        rmse_ridge_stock,
        rmse_svr_stock,
        rmse_rf_stock,
        rmse_xgb_stock,
        rmse_l,
        rmse_h
    ],
    "MAPE (%)": [
        mape_lr_stock,
        mape_ridge_stock,
        mape_svr_stock,
        mape_rf_stock,
        mape_xgb_stock,
        mape_l,
        mape_h
    ],
    "Directional Accuracy (%)": [
        dir_acc_lr_stock * 100,
        dir_acc_ridge_stock * 100,
        dir_acc_svr_stock * 100,
        dir_acc_rf_stock * 100,
        dir_acc_xgb_stock * 100,
        dir_acc_l * 100,
        dir_acc_h * 100
    ]
})

results_aapl_expanded

# ==================================
# BTC-USD model table
# ==================================

results_btc_minimal = pd.DataFrame({
    "Asset": ["BTC-USD", "BTC-USD"],
    "Model": ["Random Forest", "Hybrid RF→LSTM"],
    "MAE": [mae_rf_crypto, mae_h_btc],
    "RMSE": [rmse_rf_crypto, rmse_h_btc],
    "MAPE (%)": [mape_rf_crypto, mape_h_btc],
    "Directional Accuracy (%)": [dir_acc_rf_crypto * 100, dir_acc_h_btc * 100]
})

# ==================================
# Combined model table
# ==================================
results_combined_expanded = pd.concat(
    [results_aapl_expanded, results_btc_minimal],
    ignore_index=True
)

results_combined_expanded


In [ ]:
# ==================================
# 11.1 Comprehensive Model Comparison 
# ==================================

# Create comprehensive results table
results_comprehensive = pd.DataFrame({
    "Asset": ["AAPL"] * 9 + ["BTC-USD"] * 2,
    "Model": [
        "Linear Regression",
        "Ridge Regression",
        "SVR (RBF)",
        "Random Forest (Reg)",
        "XGBoost (Reg)",
        "LSTM (Reg)",
        "Hybrid RF→LSTM (Reg)",
        "Random Forest (Clf)",
        "LSTM (Clf)",
        "Random Forest (Reg)",
        "Hybrid RF→LSTM (Reg)"
    ],
    "Task": [
        "Regression", "Regression", "Regression", "Regression", "Regression",
        "Regression", "Regression", "Classification", "Classification",
        "Regression", "Regression"
    ],
    "MAE": [
        mae_lr_stock, mae_ridge_stock, mae_svr_stock,
        mae_rf_stock, mae_xgb_stock, mae_l, mae_h,
        np.nan, np.nan,  # Classification doesn't have MAE
        mae_rf_crypto, mae_h_btc
    ],
    "RMSE": [
        rmse_lr_stock, rmse_ridge_stock, rmse_svr_stock,
        rmse_rf_stock, rmse_xgb_stock, rmse_l, rmse_h,
        np.nan, np.nan,  # Classification doesn't have RMSE
        rmse_rf_crypto, rmse_h_btc
    ],
    "MAPE (%)": [
        mape_lr_stock, mape_ridge_stock, mape_svr_stock,
        mape_rf_stock, mape_xgb_stock, mape_l, mape_h,
        np.nan, np.nan,  # Classification doesn't have MAPE
        mape_rf_crypto, mape_h_btc
    ],
    "Accuracy/Dir_Acc (%)": [
        dir_acc_lr_stock * 100,
        dir_acc_ridge_stock * 100,
        dir_acc_svr_stock * 100,
        dir_acc_rf_stock * 100,
        dir_acc_xgb_stock * 100,
        dir_acc_l * 100,
        dir_acc_h * 100,
        acc_rf_stock * 100,  # Classification accuracy
        acc_lstm_clf * 100,  # Classification accuracy
        dir_acc_rf_crypto * 100,
        dir_acc_h_btc * 100
    ],
    "F1-Score": [
        np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
        f1_rf_stock,  # Classification F1
        f1_lstm_clf,  # Classification F1
        np.nan, np.nan
    ],
    "ROC AUC": [
        np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
        auc_rf_stock,  # Classification AUC
        auc_lstm_clf,  # Classification AUC
        np.nan, np.nan
    ]
})



In [ ]:
# Format the table for display
results_display = results_comprehensive.copy()

# Format numeric columns
results_display["MAE"] = results_display["MAE"].apply(
    lambda x: f"{x:.4f}" if pd.notna(x) else "-"
)
results_display["RMSE"] = results_display["RMSE"].apply(
    lambda x: f"{x:.4f}" if pd.notna(x) else "-"
)
results_display["MAPE (%)"] = results_display["MAPE (%)"].apply(
    lambda x: f"{x:.2f}" if pd.notna(x) else "-"
)
results_display["Accuracy/Dir_Acc (%)"] = results_display["Accuracy/Dir_Acc (%)"].apply(
    lambda x: f"{x:.2f}" if pd.notna(x) else "-"
)
results_display["F1-Score"] = results_display["F1-Score"].apply(
    lambda x: f"{x:.4f}" if pd.notna(x) else "-"
)
results_display["ROC AUC"] = results_display["ROC AUC"].apply(
    lambda x: f"{x:.4f}" if pd.notna(x) else "-"
)

print("=" * 120)
print("COMPREHENSIVE MODEL COMPARISON TABLE")
print("=" * 120)
display(results_display)

# Highlight best performers
print("\n" + "=" * 120)
print("BEST PERFORMERS BY METRIC")
print("=" * 120)

# Regression metrics (AAPL only)
reg_mask = (results_comprehensive["Asset"] == "AAPL") & (results_comprehensive["Task"] == "Regression")
reg_results = results_comprehensive[reg_mask]

best_mae = reg_results.loc[reg_results["MAE"].idxmin()]
best_rmse = reg_results.loc[reg_results["RMSE"].idxmin()]
best_mape = reg_results.loc[reg_results["MAPE (%)"].idxmin()]

print("\nRegression Task (AAPL):")
print(f"  Best MAE:  {best_mae['Model']:<25} ({best_mae['MAE']:.4f})")
print(f"  Best RMSE: {best_rmse['Model']:<25} ({best_rmse['RMSE']:.4f})")
print(f"  Best MAPE: {best_mape['Model']:<25} ({best_mape['MAPE (%)']:.2f}%)")

# Classification metrics (AAPL only)
clf_mask = (results_comprehensive["Asset"] == "AAPL") & (results_comprehensive["Task"] == "Classification")
clf_results = results_comprehensive[clf_mask]

best_acc = clf_results.loc[clf_results["Accuracy/Dir_Acc (%)"].idxmax()]
best_f1 = clf_results.loc[clf_results["F1-Score"].idxmax()]
best_auc = clf_results.loc[clf_results["ROC AUC"].idxmax()]

print("\nClassification Task (AAPL):")
print(f"  Best Accuracy: {best_acc['Model']:<25} ({best_acc['Accuracy/Dir_Acc (%)']:.2f}%)")
print(f"  Best F1-Score: {best_f1['Model']:<25} ({best_f1['F1-Score']:.4f})")
print(f"  Best ROC AUC:  {best_auc['Model']:<25} ({best_auc['ROC AUC']:.4f})")

print("=" * 120)


In [ ]:
# Create visualization comparing regression vs classification
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: MAE Comparison (Regression only)
reg_models = reg_results["Model"].str.replace(" (Reg)", "", regex=False)
mae_values = reg_results["MAE"].values
axes[0, 0].barh(reg_models, mae_values, color='steelblue')
axes[0, 0].set_xlabel('MAE')
axes[0, 0].set_title('Mean Absolute Error (AAPL - Regression Models)')
axes[0, 0].invert_yaxis()
axes[0, 0].grid(axis='x', alpha=0.3)

# Highlight best
best_idx = np.argmin(mae_values)
axes[0, 0].get_children()[best_idx].set_color('darkgreen')

# Plot 2: Directional Accuracy (Regression)
dir_acc_values = reg_results["Accuracy/Dir_Acc (%)"].values
axes[0, 1].barh(reg_models, dir_acc_values, color='coral')
axes[0, 1].axvline(x=50, color='red', linestyle='--', label='Random (50%)', alpha=0.7)
axes[0, 1].set_xlabel('Directional Accuracy (%)')
axes[0, 1].set_title('Directional Accuracy (AAPL - Regression Models)')
axes[0, 1].invert_yaxis()
axes[0, 1].legend()
axes[0, 1].grid(axis='x', alpha=0.3)

# Plot 3: Classification Accuracy
clf_models = clf_results["Model"].str.replace(" (Clf)", "", regex=False)
clf_acc_values = clf_results["Accuracy/Dir_Acc (%)"].values
axes[1, 0].barh(clf_models, clf_acc_values, color='green')
axes[1, 0].axvline(x=50, color='red', linestyle='--', label='Random (50%)', alpha=0.7)
axes[1, 0].set_xlabel('Classification Accuracy (%)')
axes[1, 0].set_title('Classification Accuracy (AAPL - Classification Models)')
axes[1, 0].invert_yaxis()
axes[1, 0].legend()
axes[1, 0].grid(axis='x', alpha=0.3)

# Highlight best
best_clf_idx = np.argmax(clf_acc_values)
axes[1, 0].get_children()[best_clf_idx].set_color('darkgreen')

# Plot 4: ROC AUC Comparison
auc_values = clf_results["ROC AUC"].values
axes[1, 1].barh(clf_models, auc_values, color='purple')
axes[1, 1].axvline(x=0.5, color='red', linestyle='--', label='Random (0.5)', alpha=0.7)
axes[1, 1].set_xlabel('ROC AUC')
axes[1, 1].set_title('ROC AUC Score (AAPL - Classification Models)')
axes[1, 1].set_xlim([0, 1])
axes[1, 1].invert_yaxis()
axes[1, 1].legend()
axes[1, 1].grid(axis='x', alpha=0.3)

# Highlight best
best_auc_idx = np.argmax(auc_values)
axes[1, 1].get_children()[best_auc_idx].set_color('darkgreen')

plt.tight_layout()
savefig("comprehensive_model_comparison.png")
plt.show()

### 11.2 Relative Improvement Analysis

This section quantifies the relative improvement of the hybrid model over baseline approaches using percentage-based metrics.


In [ ]:
# ==================================
# 11.2 Relative improvement (%) vs Random Forest
# ==================================

improvement = pd.DataFrame({
    "Asset": ["AAPL", "BTC-USD"],
    "MAE Improvement (%)": [
        (1 - mae_h / mae_rf_stock) * 100,
        (1 - mae_h_btc / mae_rf_crypto) * 100
    ],
    "RMSE Improvement (%)": [
        (1 - rmse_h / rmse_rf_stock) * 100,
        (1 - rmse_h_btc / rmse_rf_crypto) * 100
    ]
})

improvement


### 11.3 Visual Comparison of Hybrid Model Gains

A bar chart visualises the performance gains achieved by the hybrid model across different asset classes.


In [ ]:
# ==================================
# 11.3 Visual comparison of hybrid gains
# ==================================

improvement.set_index("Asset").plot(
    kind="bar",
    figsize=(7, 4)
)

plt.title("Hybrid RF→LSTM Improvement over Random Forest")
plt.ylabel("Improvement (%)")
plt.xticks(rotation=0)
plt.tight_layout()
savefig("hybrid_improvement_rf.png")
plt.show()


## 11.4 Robust evaluation additions (naïve baseline & walk-forward validation)

To strengthen the validity of the findings for a time-series forecasting task, this section adds two widely used evaluation components:

- **Naïve (persistence) baseline:** tomorrow's close is assumed to be today's close. This is a strong baseline in financial time series and helps contextualise model performance.
- **Walk-forward (TimeSeriesSplit) validation:** classical ML models are evaluated over multiple chronological folds to reduce dependence on a single train/test split.

> Note: Deep learning models (LSTM / Hybrid) can be computationally expensive to re-train across many folds. Therefore, walk-forward validation is applied to the classical ML baselines, while LSTM/Hybrid results are reported on the fixed holdout split used throughout the notebook.


In [ ]:
# ==================================
# 11.4.1 Utility: unified metrics + naïve baseline
# ==================================
from sklearn.model_selection import TimeSeriesSplit
from sklearn.base import clone
from sklearn.pipeline import Pipeline

def regression_metrics_dict(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # avoid division by zero in MAPE
    denom = np.where(np.abs(y_true) < 1e-9, 1e-9, np.abs(y_true))
    mape = np.mean(np.abs((y_true - y_pred) / denom)) * 100
    dir_acc = directional_accuracy(pd.Series(y_true), pd.Series(y_pred))
    return {
        "MAE": mae,
        "RMSE": rmse,
        "MAPE_%": mape,
        "Directional_Acc": dir_acc
    }

def naive_persistence_forecast(y_train, y_test):
    """Predicts y[t] as y[t-1]. For the first test point, uses last train value."""
    y_train = pd.Series(y_train).reset_index(drop=True)
    y_test = pd.Series(y_test).reset_index(drop=True)
    y_pred = y_test.shift(1)
    y_pred.iloc[0] = y_train.iloc[-1]
    return y_pred.values

# Naïve baseline (AAPL)
y_pred_naive_stock = naive_persistence_forecast(y_train_stock, y_test_stock)
naive_stock_metrics = regression_metrics_dict(y_test_stock, y_pred_naive_stock)

print("Naïve (Persistence) Baseline – AAPL")
for k,v in naive_stock_metrics.items():
    if k == "Directional_Acc":
        print(f"{k}: {v:.2%}")
    elif k == "MAPE_%":
        print(f"{k}: {v:.2f}%")
    else:
        print(f"{k}: {v:.4f}")

# Naïve baseline (BTC)
y_pred_naive_crypto = naive_persistence_forecast(y_train_crypto, y_test_crypto)
naive_crypto_metrics = regression_metrics_dict(y_test_crypto, y_pred_naive_crypto)

print("\nNaïve (Persistence) Baseline – BTC-USD")
for k,v in naive_crypto_metrics.items():
    if k == "Directional_Acc":
        print(f"{k}: {v:.2%}")
    elif k == "MAPE_%":
        print(f"{k}: {v:.2f}%")
    else:
        print(f"{k}: {v:.4f}")


In [ ]:
# ==================================
# 11.4.2 Walk-forward validation for classical ML baselines
# ==================================
# This evaluates each model over multiple chronological folds.
# It is especially useful for financial time series where regime changes occur.
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.exceptions import ConvergenceWarning
from scipy.linalg import LinAlgWarning
import warnings

tscv = TimeSeriesSplit(n_splits=5)

def walk_forward_cv(model, X_df, y_series, name="model"):
    fold_rows = []
    for fold, (train_idx, test_idx) in enumerate(tscv.split(X_df), start=1):
        X_tr, X_te = X_df.iloc[train_idx], X_df.iloc[test_idx]
        y_tr, y_te = y_series.iloc[train_idx], y_series.iloc[test_idx]

        m = clone(model)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=LinAlgWarning)
            m.fit(X_tr, y_tr)

        y_pred = m.predict(X_te)
        metrics = regression_metrics_dict(y_te, y_pred)
        metrics["Fold"] = fold
        fold_rows.append(metrics)

    df = pd.DataFrame(fold_rows)
    summary = df.drop(columns=["Fold"]).mean(numeric_only=True).to_dict()
    print(f"{name} walk-forward CV (mean over {tscv.get_n_splits()} folds)")
    for k, v in summary.items():
        if k == "Directional_Acc":
            print(f"  {k}: {v:.2%}")
        elif k == "MAPE_%":
            print(f"  {k}: {v:.2f}%")
        else:
            print(f"  {k}: {v:.4f}")
    return df, summary


# Define baseline models for CV
baseline_models = {
    "LinearRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("lr", LinearRegression())
    ]),
    "Ridge": Pipeline([
        ("scaler", StandardScaler()),
        ("ridge", Ridge(alpha=10.0))  # slightly stronger regularisation improves stability
    ]),
    "RandomForest": RandomForestRegressor(
        n_estimators=300, random_state=SEED, n_jobs=-1, max_depth=None, min_samples_leaf=1
    ),
    "SVR(RBF)": Pipeline([
        ("scaler", StandardScaler()),
        ("svr", SVR(kernel="rbf", C=50, gamma="scale", epsilon=0.1))
    ])
}

if XGB_AVAILABLE:
    baseline_models["XGBoost"] = XGBRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=5,
        subsample=0.8, colsample_bytree=0.8,
        random_state=SEED, n_jobs=-1
    )

# Run walk-forward CV for AAPL and BTC
cv_results = {}

print("\n--- Walk-forward CV: AAPL ---")
for name, model in baseline_models.items():
    df_folds, summary = walk_forward_cv(model, X, y_stock, name)
    cv_results[("AAPL", name)] = summary

print("\n--- Walk-forward CV: BTC-USD ---")
for name, model in baseline_models.items():
    df_folds, summary = walk_forward_cv(model, X, y_crypto, name)
    cv_results[("BTC", name)] = summary


### 11.4.3 Compact summary table (naïve baseline + walk-forward CV)

The table below combines:
- the **naïve baseline** performance on the holdout split, and
- the **walk-forward CV mean** scores for classical ML models.

Use this table in Chapter 4 (Results) to support stronger, more defensible comparisons.


In [ ]:
# Build summary table
rows = []

# Naïve baselines
rows.append({"Asset": "AAPL", "Model": "Naïve (Persistence)", **naive_stock_metrics})
rows.append({"Asset": "BTC", "Model": "Naïve (Persistence)", **naive_crypto_metrics})

# CV means
for (asset, name), summary in cv_results.items():
    rows.append({"Asset": asset, "Model": f"{name} (CV mean)", **summary})

summary_df = pd.DataFrame(rows)
summary_df["Directional_Acc"] = summary_df["Directional_Acc"].apply(lambda x: f"{x:.2%}")
summary_df["MAE"] = summary_df["MAE"].map(lambda x: f"{x:.4f}")
summary_df["RMSE"] = summary_df["RMSE"].map(lambda x: f"{x:.4f}")
summary_df["MAPE_%"] = summary_df["MAPE_%"].map(lambda x: f"{x:.2f}%")

display(summary_df)


In [ ]:
# ==================================
# 11.5 Market Regime Analysis (NEW)
# ==================================

print("=" * 70)
print("MARKET REGIME ANALYSIS")
print("=" * 70)

def analyze_by_market_regime(y_true, y_pred, prices, window=20):
    """
    Evaluates model performance in different market conditions.
    
    Parameters:
    -----------
    y_true : array-like
        True values
    y_pred : array-like
        Predicted values
    prices : array-like
        Actual prices for regime detection
    window : int
        Window for calculating trend
    
    Returns:
    --------
    dict : Performance metrics by regime
    """
    # Calculate price momentum
    returns = pd.Series(prices).pct_change()
    momentum = returns.rolling(window).mean()
    
    # Define regimes
    bull_mask = (momentum > 0.001).values  # Uptrend
    bear_mask = (momentum < -0.001).values  # Downtrend
    sideways_mask = (~bull_mask & ~bear_mask)  # Sideways
    
    # Remove NaN from rolling calculation
    valid_idx = ~momentum.isna()
    
    def calc_metrics(mask):
        mask_valid = mask & valid_idx
        if mask_valid.sum() < 5:  # Need at least 5 samples
            return None
        
        y_t = y_true[mask_valid]
        y_p = y_pred[mask_valid]
        
        mae = mean_absolute_error(y_t, y_p)
        rmse = np.sqrt(mean_squared_error(y_t, y_p))
        mape = np.mean(np.abs((y_t - y_p) / (y_t + 1e-9))) * 100
        dir_acc = directional_accuracy(pd.Series(y_t), pd.Series(y_p))
        
        return {
            'Count': mask_valid.sum(),
            'MAE': mae,
            'RMSE': rmse,
            'MAPE (%)': mape,
            'Dir_Acc (%)': dir_acc * 100
        }
    
    bull_metrics = calc_metrics(bull_mask)
    bear_metrics = calc_metrics(bear_mask)
    sideways_metrics = calc_metrics(sideways_mask)
    
    return {
        'Bull Market': bull_metrics,
        'Bear Market': bear_metrics,
        'Sideways Market': sideways_metrics,
        'Bull %': (bull_mask & valid_idx).sum() / valid_idx.sum() * 100,
        'Bear %': (bear_mask & valid_idx).sum() / valid_idx.sum() * 100,
        'Sideways %': (sideways_mask & valid_idx).sum() / valid_idx.sum() * 100
    }

# Analyze Hybrid RF→LSTM (Regression)
print("\n" + "=" * 70)
print("HYBRID RF→LSTM - MARKET REGIME ANALYSIS (AAPL)")
print("=" * 70)

regime_results = analyze_by_market_regime(
    y_true=y_true_h,
    y_pred=y_pred_h,
    prices=y_true_h,
    window=20
)

print(f"\nMarket Distribution:")
print(f"  Bull Market:     {regime_results['Bull %']:.1f}%")
print(f"  Bear Market:     {regime_results['Bear %']:.1f}%")
print(f"  Sideways Market: {regime_results['Sideways %']:.1f}%")

regime_df = pd.DataFrame({
    'Regime': ['Bull Market', 'Bear Market', 'Sideways Market'],
    'Sample Count': [
        regime_results['Bull Market']['Count'],
        regime_results['Bear Market']['Count'],
        regime_results['Sideways Market']['Count']
    ],
    'MAE': [
        regime_results['Bull Market']['MAE'],
        regime_results['Bear Market']['MAE'],
        regime_results['Sideways Market']['MAE']
    ],
    'RMSE': [
        regime_results['Bull Market']['RMSE'],
        regime_results['Bear Market']['RMSE'],
        regime_results['Sideways Market']['RMSE']
    ],
    'MAPE (%)': [
        regime_results['Bull Market']['MAPE (%)'],
        regime_results['Bear Market']['MAPE (%)'],
        regime_results['Sideways Market']['MAPE (%)']
    ],
    'Dir_Acc (%)': [
        regime_results['Bull Market']['Dir_Acc (%)'],
        regime_results['Bear Market']['Dir_Acc (%)'],
        regime_results['Sideways Market']['Dir_Acc (%)']
    ]
})

print("\nPerformance by Market Regime:")
display(regime_df)


In [ ]:
# Visualize regime performance
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: MAE by Regime
regimes = ['Bull', 'Bear', 'Sideways']
mae_by_regime = [
    regime_results['Bull Market']['MAE'],
    regime_results['Bear Market']['MAE'],
    regime_results['Sideways Market']['MAE']
]
axes[0, 0].bar(regimes, mae_by_regime, color=['green', 'red', 'gray'])
axes[0, 0].set_ylabel('MAE')
axes[0, 0].set_title('MAE by Market Regime')
axes[0, 0].grid(axis='y', alpha=0.3)

# Plot 2: RMSE by Regime
rmse_by_regime = [
    regime_results['Bull Market']['RMSE'],
    regime_results['Bear Market']['RMSE'],
    regime_results['Sideways Market']['RMSE']
]
axes[0, 1].bar(regimes, rmse_by_regime, color=['green', 'red', 'gray'])
axes[0, 1].set_ylabel('RMSE')
axes[0, 1].set_title('RMSE by Market Regime')
axes[0, 1].grid(axis='y', alpha=0.3)

# Plot 3: MAPE by Regime
mape_by_regime = [
    regime_results['Bull Market']['MAPE (%)'],
    regime_results['Bear Market']['MAPE (%)'],
    regime_results['Sideways Market']['MAPE (%)']
]
axes[1, 0].bar(regimes, mape_by_regime, color=['green', 'red', 'gray'])
axes[1, 0].set_ylabel('MAPE (%)')
axes[1, 0].set_title('MAPE by Market Regime')
axes[1, 0].grid(axis='y', alpha=0.3)

# Plot 4: Directional Accuracy by Regime
dir_acc_by_regime = [
    regime_results['Bull Market']['Dir_Acc (%)'],
    regime_results['Bear Market']['Dir_Acc (%)'],
    regime_results['Sideways Market']['Dir_Acc (%)']
]
axes[1, 1].bar(regimes, dir_acc_by_regime, color=['green', 'red', 'gray'])
axes[1, 1].axhline(y=50, color='black', linestyle='--', label='Random (50%)', alpha=0.7)
axes[1, 1].set_ylabel('Directional Accuracy (%)')
axes[1, 1].set_title('Directional Accuracy by Market Regime')
axes[1, 1].legend()
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
savefig("market_regime_analysis.png")
plt.show()

# Key insights
print("\n" + "=" * 70)
print("KEY INSIGHTS")
print("=" * 70)

best_regime_mae = regimes[np.argmin(mae_by_regime)]
worst_regime_mae = regimes[np.argmax(mae_by_regime)]
best_regime_dir = regimes[np.argmax(dir_acc_by_regime)]

print(f"✓ Best MAE performance:        {best_regime_mae} Market ({min(mae_by_regime):.4f})")
print(f"✗ Worst MAE performance:       {worst_regime_mae} Market ({max(mae_by_regime):.4f})")
print(f"✓ Best directional accuracy:   {best_regime_dir} Market ({max(dir_acc_by_regime):.2f}%)")
print(f"  Model performs {max(mae_by_regime)/min(mae_by_regime):.2f}x worse in {worst_regime_mae} markets")
print("=" * 70)

In [ ]:
# ==================================
# 11.6 Statistical Significance Testing 
# ==================================

import numpy as np
import pandas as pd
from scipy.stats import ttest_rel, wilcoxon

def test_model_significance(y_true, y_pred1, y_pred2,
                            model1_name="Model 1",
                            model2_name="Model 2"):
    """
    Paired statistical comparison of two regression models
    using absolute prediction errors.
    """

    y_true = np.asarray(y_true).ravel()
    y_pred1 = np.asarray(y_pred1).ravel()
    y_pred2 = np.asarray(y_pred2).ravel()

    assert len(y_true) == len(y_pred1) == len(y_pred2), \
        "Arrays must be aligned and of equal length."

    # Absolute errors (paired by time index)
    err1 = np.abs(y_true - y_pred1)
    err2 = np.abs(y_true - y_pred2)

    # Paired t-test
    t_stat, p_t = ttest_rel(err1, err2)

    # Wilcoxon signed-rank test (robust alternative)
    try:
        w_stat, p_w = wilcoxon(err1, err2)
    except ValueError:
        p_w = np.nan

    # Effect size (Cohen's d for paired samples)
    diff = err1 - err2
    cohens_d = diff.mean() / (diff.std(ddof=1) + 1e-12)

    print("\n" + "=" * 70)
    print(f"STATISTICAL SIGNIFICANCE TEST: {model1_name} vs {model2_name}")
    print("=" * 70)
    print(f"Mean absolute error ({model1_name}): {err1.mean():.4f}")
    print(f"Mean absolute error ({model2_name}): {err2.mean():.4f}")
    print(f"Mean difference (err1 - err2): {diff.mean():.4f}")
    print(f"Paired t-test p-value: {p_t:.6f}")
    print(f"Wilcoxon p-value: {p_w:.6f}" if not np.isnan(p_w) else "Wilcoxon p-value: N/A")
    print(f"Cohen’s d (paired): {cohens_d:.4f}")

    if p_t < 0.05:
        print("Interpretation: Difference is statistically significant at α = 0.05.")
    else:
        print("Interpretation: Difference is not statistically significant at α = 0.05.")

    return {
        "Comparison": f"{model1_name} vs {model2_name}",
        "MAE_" + model1_name: err1.mean(),
        "MAE_" + model2_name: err2.mean(),
        "Mean_Diff": diff.mean(),
        "p_ttest": p_t,
        "p_wilcoxon": p_w,
        "cohens_d": cohens_d
    }

# Get the last 477 predictions from RF to match hybrid
rf_predictions_aligned = y_pred_rf_stock[-len(y_true_h):]
y_test_aligned = y_test_stock.values[-len(y_true_h):]

# -------------------------
# Test 1: Random Forest vs Hybrid
# -------------------------
sig_test1 = test_model_significance(
    y_true=y_true_h,
    y_pred1=rf_predictions_aligned,
    y_pred2=y_pred_h,
    model1_name="Random Forest",
    model2_name="Hybrid RF→LSTM"
)

# -------------------------
# Test 2: LSTM vs Hybrid 
# -------------------------
sig_test2 = test_model_significance(
    y_true=y_true_lstm,
    y_pred1=y_pred_lstm,
    y_pred2=y_pred_h,
    model1_name="LSTM",
    model2_name="Hybrid RF→LSTM"
)

# Summary table 
sig_results_df = pd.DataFrame([sig_test1, sig_test2])
sig_results_df


## 12. Conclusion

This project investigated the effectiveness of machine learning and deep learning models for predicting stock and cryptocurrency prices using publicly available financial data. Using Apple Inc. (AAPL) as a representative stock and Bitcoin (BTC-USD) as a representative cryptocurrency, multiple models were implemented and evaluated including Random Forest, XGBoost, Support Vector Regression, LSTM and a hybrid Random Forest–LSTM approach.

The experimental results showed that traditional machine learning models such as Random Forest and XGBoost were able to capture basic price-level patterns but struggled to consistently predict short-term market direction. The LSTM model which incorporates temporal dependencies, demonstrated improved ability to follow overall price trends. The proposed hybrid approach which combines Random Forest-based feature selection with LSTM sequence learning, achieved better error-based performance metrics such as MAE, RMSE and MAPE, particularly for the cryptocurrency dataset.

However, directional accuracy remained relatively low across all models, highlighting the inherent difficulty of short-term trend prediction in highly volatile financial markets. Overall, the findings suggest that hybrid models can improve numerical prediction accuracy, but reliable trend prediction remains a challenging task especially for cryptocurrencies.


## 13. Limitations

Despite achieving results, this project has several limitations that should be acknowledged. First, the models were trained on historical price data and technical indicators only. External factors such as macroeconomic events, news sentiment and geopolitical influences were not considered, even though they can have a significant impact on financial markets.

Second, the evaluation was based on a single train–test split rather than a full walk-forward or rolling-window validation approach. While this simplifies the experimental design and reduces computational cost, it may limit the robustness of the conclusions.

Third, directional accuracy remained relatively low, indicating that the models were better suited for price-level regression than for precise up/down trend prediction. This is a known challenge in financial forecasting, especially for highly volatile assets such as cryptocurrencies.

Finally, the LSTM architecture used in this study was intentionally kept simple to maintain interpretability and computational feasibility.


## 14. Future Work

Future research could extend this work in several directions. One possible improvement would be to incorporate additional data sources such as macroeconomic indicators from FRED, company fundamentals or news and sentiment analysis to capture external market drivers.

Another extension would be to adopt a rolling-window or walk-forward validation strategy to better simulate real-world forecasting conditions and improve the robustness of model evaluation. Hyperparameter optimisation techniques such as grid search or Bayesian optimisation could also be applied to further improve model performance.

More advanced deep learning architectures such as bidirectional LSTMs, GRU networks or attention-based models could be explored to better capture complex temporal dependencies. Additionally, reframing the problem as a classification task focused specifically on trend direction rather than price regression may improve directional accuracy.
